<a href="https://colab.research.google.com/github/Anudeep27-bit/Iirm-fin-acct-dp-prod.IYR_Process/blob/main/IYR_process_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import and install requirements

In [ ]:
from google.cloud import bigquery

import pandas as pd
from io import BytesIO
from datetime import datetime
pd.options.display.max_columns = None
import pandas_gbq
import os

import gspread
import numpy as np

from google.colab import auth
from google.auth import default
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print('Authenticated')

from google.colab import drive
drive.mount('/content/drive')

Authenticated
Mounted at /content/drive


In [ ]:
def query_bq(query:str,project_id = 'irm-fin-acct-dp-prod'):
    """
    project_id: Set project_id to your Google Cloud Platform project ID.
    ox"
    """

    with bigquery.Client(project=project_id) as client:
        query_job = client.query(query)
        df = query_job.to_dataframe()
    return df

def upload_df_bq(df:pd.DataFrame,project_id,table_id):
    """

    project_id: Set project_id to your Google Cloud Platform project ID.
    ox"

    table_id: Set table_id to the full destination table ID (including the
           dataset ID)."""
    pandas_gbq.to_gbq(df, table_id, project_id=project_id,if_exists='replace')

## Declare functions to download and upload data to BQ

## Process

### Get Assumptions and upload to BQ

In [ ]:
Ass = pd.read_excel('/content/drive/MyDrive/IYR Online Files/B2B Assumptions Lookup.xlsx',keep_default_na=False,na_values=['', '#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#IND', '1.#QNAN', '<NA>', 'N/A', 'NULL', 'NaN', 'None', 'n/a', 'nan', 'null'])
Ass.columns = [x.replace(' ','_') for x in Ass.columns ]
upload_df_bq(Ass,'irm-fin-acct-dp-prod','IYR_Process.B2B_Assumptions_In')

### Get E2E and upload to BQ

In [ ]:
files = os.listdir('/content/drive/Shareddrives/IYR Sandbox/IYR Live/Data Sources/E2E')
excel_files = list(filter(lambda x: x[-4:] == 'xlsx',files))
times = list(map(lambda x: os.path.getmtime(f'/content/drive/Shareddrives/IYR Sandbox/IYR Live/Data Sources/E2E/{x}'),excel_files))
last_time = max(times)
last_file = excel_files[times.index(last_time)]
print(last_file)

E2E = pd.read_excel(f'/content/drive/Shareddrives/IYR Sandbox/IYR Live/Data Sources/E2E/{last_file}')

E2E Forecast Report 2024-2025 - July 17th.xlsx


In [ ]:
E2E.columns = [x.replace('--','_').replace(' ','_') for x in E2E.columns]

In [ ]:
E2E['Customer_ID'] = E2E['Customer_ID'].astype('str')
E2E["Project"] = E2E["Project"].astype('str')
E2E["SalesForce_ID"] = E2E["SalesForce_ID"].astype('str')
E2E["Region"] = E2E["Region"].fillna('NA')

In [ ]:
E2E['TOTAL'].sum()

396378462

In [ ]:
E2E.groupby(["Region"]).size()

Region
APAC        502
EUR        1434
LATAM      1887
MENATSA       6
NA         1049
dtype: int64

In [ ]:
upload_df_bq(E2E,"irm-fin-acct-dp-prod",'IYR_Process.E2E_ln')

100%|██████████| 1/1 [00:00<00:00, 843.58it/s]


### Template process

In [ ]:
#CREATE OR REPLACE TABLE irm-fin-acct-dp-prod.IYR_Process.B2B_In_Snap AS
#(SELECT * FROM irm-fin-acct-dp-prod.IYR_Process.B2B_In)

In [ ]:
_ = query_bq('''
UPDATE irm-fin-acct-dp-prod.IYR_Process.B2B_In_Snap
SET Opportunity_ID = '0064Q00001iafiR1'
WHERE Opportunity_ID = '0064Q00001iafiR'
''')

_ = query_bq('''
UPDATE irm-fin-acct-dp-prod.IYR_Process.B2B_In_Snap
SET Opportunity_ID = '0064Q00001iafir2'
WHERE Opportunity_ID = '0064Q00001iafir'
''')

In [ ]:
_ = query_bq('''
CREATE OR REPLACE TABLE irm-fin-acct-dp-prod.IYR_Process.Prod_Digi_Nw AS (
WITH BU_Lookup AS(
SELECT DISTINCT Product,BU
FROM irm-fin-acct-dp-prod.IYR_Process.CC_Lookups),

ProductCleanse AS (SELECT DISTINCT ProdF,ProdR
FROM irm-fin-acct-dp-prod.IYR_Process.CC_Cleanse WHERE ProdF <> ''),

PIDMapping AS (SELECT DISTINCT Prod_ClassLineName,Prod_Digital_New,Prod_Digital_Hist
FROM irm-fin-acct-dp-prod.IYR_Process.CC_Lookups
WHERE Prod_ClassLineName IS NOT NULL AND Prod_ClassLineName <> ''),

FILTER_B2B AS (
SELECT *
FROM irm-fin-acct-dp-prod.IYR_Process.B2B_In_Snap
WHERE Product_Class <> 'IMDC'
AND  Product_Class <> 'ITRenew'
AND Product_Class <> 'ALM'
AND Forecast_Category <> 'Omitted'
AND Total_Annual_Amount <> 0.0
AND (Stage <> '1 - Qualification' OR (Cluster = 'EMEA: MENATSA' AND Product_Class <> 'MAS'))
AND (
  (Forecast_Revenue_Date >= CAST('2024-01-01' AS DATE) AND Forecast_Revenue_Date < CAST('2026-01-01' AS DATE))
  OR
  (Close_Date >= CAST('2023-01-01' AS DATE) AND Close_Date < CAST('2026-01-01' AS DATE))
  OR
  (Opportunity_ID IN (SELECT OppID FROM irm-fin-acct-dp-prod.IYR_Process.CC_2022OppIDS))
    )
),

CTE AS (SELECT Opportunity_ID AS Opp_ID,
Opportunity_Name,
Account_Name,
CASE WHEN Product_Class = 'IMES' THEN 'IMES' ELSE Region END AS Region,
Country,
CASE WHEN Cluster = 'Canada' THEN 'NA: Canada' WHEN LEFT(Cluster,2) = 'US' THEN REPLACE(Cluster,'US:','NA:') ELSE Cluster END AS Cluster,
CASE WHEN Product_Class = 'Iron Cloud' THEN 'Cloud' ELSE Product_Class END AS Product,
CASE WHEN Commission_Category = 'M1' THEN 'Storage' ELSE 'Service' END AS StoSer,
CASE WHEN Segment = 'Global Industries' THEN 'GI' ELSE 'Region' END AS Segment,
CASE WHEN Revenue_Type = 'Non Recurring' THEN 'Non-recurring' ELSE Revenue_Type END AS RNR,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
CASE WHEN Product_Revenue_Band IN ("$80k - $250k" , "$25k - $80k") THEN "$25k - $250k" ELSE Product_Revenue_Band END AS Deal_Size,
CASE
WHEN SAFE_CAST(LEFT(Stage,1) AS INTEGER)<= 4  OR (Stage = 'Pipeline') THEN 'Pipeline'
WHEN (SAFE_CAST(LEFT(Stage,1) AS INTEGER)>= 5  AND SAFE_CAST(LEFT(Stage,1) AS INTEGER)<= 7) OR (Stage = 'Bookings') THEN 'Bookings'
ELSE NULL END AS Phase,
Stage,
Forecast_Category,
Forecast_Type,
Probability,
Age,
Created_Date,
Close_Date,
CASE WHEN REGION = 'NA' AND Close_Date IS NOT NULL AND First_Bill_Date IS NULL THEN DATE_ADD(Close_Date, INTERVAL 90 DAY) ELSE First_Bill_Date END AS First_Bill_Date,
Forecast_First_Bill_Date,
Forecasted_Billings,
Booking_Forecast,
Total_Annual_Amount AS Annual_Amt,
Forecast_Total,
Forecast_Revenue_Date,
IFNULL(Pipeline_Forecast,0) AS Pipeline_Forecast,
IFNULL(Sales_Forecast,0) AS Sales_Forecast,
IFNULL(Ops_Forecast,0) AS Ops_Forecast,
IFNULL(Booked_Assumptions,0) AS Booked_Assumptions,
IFNULL(Actual_Billings,0) AS Actual_Billings,
CURRENT_TIMESTAMP() AS Timestamp,
'B2B' AS Data_Source,
IFNULL(Pipeline_Forecast,0)+IFNULL(Sales_Forecast,0)+IFNULL(Ops_Forecast,0)+IFNULL(Booked_Assumptions,0)+IFNULL(Actual_Billings,0) AS Value,
CAST(FORMAT_DATETIME('%b_%y',Forecast_Revenue_Date) AS STRING) AS Forecast_pd,
CASE WHEN Product_Class = 'Iron Cloud' THEN 'Digital' ELSE BU_Lookup.BU END AS BU,
CONCAT(Product_Class,Product_Line,Product_Name) AS PID
FROM FILTER_B2B LEFT JOIN BU_Lookup ON Product_Class = BU_Lookup.Product LEFT JOIN ProductCleanse ON Product=ProductCleanse.ProdF),

CREATE_UID AS(
SELECT *,CONCAT(Opp_ID,'-',Country,'-',Product,'-',StoSer,'-',RNR,'-',Deal_Size) AS UID,
FROM CTE
WHERE BU = 'Digital')
SELECT DISTINCT Region,Cluster,Deal_Size,Country,BU,Product,StoSer,RNR,Segment,UID,PID,Prod_Digital_New,Prod_Digital_Hist
FROM CREATE_UID LEFT JOIN PIDMapping ON Prod_ClassLineName = PID)
''')

#### B2B_Clean

In [ ]:
_ = query_bq('''CREATE OR REPLACE TABLE irm-fin-acct-dp-prod.IYR_Process.B2B_Clean AS (
WITH BU_Lookup AS(
SELECT DISTINCT Product,BU
FROM irm-fin-acct-dp-prod.IYR_Process.CC_Lookups),

ProductCleanse AS (SELECT DISTINCT ProdF,ProdR
FROM irm-fin-acct-dp-prod.IYR_Process.CC_Cleanse WHERE ProdF <> ''),

FILTER_B2B AS (
SELECT *
FROM irm-fin-acct-dp-prod.IYR_Process.B2B_In_Snap
WHERE Product_Class <> 'IMDC'
AND  Product_Class <> 'ITRenew'
AND Product_Class <> 'ALM'
AND Forecast_Category <> 'Omitted'
AND Total_Annual_Amount <> 0.0
AND (Stage <> '1 - Qualification' OR (Cluster = 'EMEA: MENATSA' AND Product_Class <> 'MAS'))
AND (
  (Forecast_Revenue_Date >= CAST('2024-01-01' AS DATE) AND Forecast_Revenue_Date < CAST('2026-01-01' AS DATE))
  OR
  (Close_Date >= CAST('2023-01-01' AS DATE) AND Close_Date < CAST('2026-01-01' AS DATE))
  OR
  (Opportunity_ID IN (SELECT OppID FROM irm-fin-acct-dp-prod.IYR_Process.CC_2022OppIDS))
    )
),

CTE AS (SELECT Opportunity_ID AS Opp_ID,
Opportunity_Name,
Account_Name,
CASE WHEN Product_Class = 'IMES' THEN 'IMES' ELSE Region END AS Region,
Country,
CASE WHEN Cluster = 'Canada' THEN 'NA: Canada' WHEN LEFT(Cluster,2) = 'US' THEN REPLACE(Cluster,'US:','NA:') ELSE Cluster END AS Cluster,
CASE WHEN Product_Class = 'Iron Cloud' THEN 'Cloud' ELSE Product_Class END AS Product,
CASE WHEN Commission_Category = 'M1' THEN 'Storage' ELSE 'Service' END AS StoSer,
CASE WHEN Segment = 'Global Industries' THEN 'GI' ELSE 'Region' END AS Segment,
CASE WHEN Revenue_Type = 'Non Recurring' THEN 'Non-recurring' ELSE Revenue_Type END AS RNR,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
CASE WHEN Product_Revenue_Band IN ("$80k - $250k" , "$25k - $80k") THEN "$25k - $250k" ELSE Product_Revenue_Band END AS Deal_Size,
CASE
WHEN SAFE_CAST(LEFT(Stage,1) AS INTEGER)<= 4  OR (Stage = 'Pipeline') THEN 'Pipeline'
WHEN (SAFE_CAST(LEFT(Stage,1) AS INTEGER)>= 5  AND SAFE_CAST(LEFT(Stage,1) AS INTEGER)<= 7) OR (Stage = 'Bookings') THEN 'Bookings'
ELSE NULL END AS Phase,
Stage,
Forecast_Category,
Forecast_Type,
Probability,
Age,
Created_Date,
Close_Date,
CASE WHEN REGION = 'NA' AND Close_Date IS NOT NULL AND First_Bill_Date IS NULL THEN DATE_ADD(Close_Date, INTERVAL 90 DAY) ELSE First_Bill_Date END AS First_Bill_Date,
Forecast_First_Bill_Date,
Forecasted_Billings,
Booking_Forecast,
Total_Annual_Amount AS Annual_Amt,
Forecast_Total,
Forecast_Revenue_Date,
IFNULL(Pipeline_Forecast,0) AS Pipeline_Forecast,
IFNULL(Sales_Forecast,0) AS Sales_Forecast,
IFNULL(Ops_Forecast,0) AS Ops_Forecast,
IFNULL(Booked_Assumptions,0) AS Booked_Assumptions,
IFNULL(Actual_Billings,0) AS Actual_Billings,
CURRENT_TIMESTAMP() AS Timestamp,
'B2B' AS Data_Source,
IFNULL(Pipeline_Forecast,0)+IFNULL(Sales_Forecast,0)+IFNULL(Ops_Forecast,0)+IFNULL(Booked_Assumptions,0)+IFNULL(Actual_Billings,0) AS Value,
CAST(FORMAT_DATETIME('%b_%y',Forecast_Revenue_Date) AS STRING) AS Forecast_pd,
CASE WHEN Product_Class = 'Iron Cloud' THEN 'Digital' ELSE BU_Lookup.BU END AS BU,
FROM FILTER_B2B LEFT JOIN BU_Lookup ON Product_Class = BU_Lookup.Product LEFT JOIN ProductCleanse ON Product=ProductCleanse.ProdF),

CREATE_UID AS(
SELECT *,CONCAT(Opp_ID,'-',Country,'-',Product,'-',StoSer,'-',RNR,'-',Deal_Size) AS UID --CONCAT(Opp_ID,'-',Country,'-',Product,'-',StoSer,'-',Segment,'-',RNR,'-',Deal_Size) AS UID
FROM CTE),

CTE3 AS (
SELECT Region, Cluster, Country,    BU, Product,    StoSer, Segment,RNR, Phase,Stage, UID, Opp_ID, Opportunity_Name, Deal_Size, Age, Account_Name, GI_Vertical,  Deal_Type, Salesforce_URL, Opportunity_Owner, Probability, Created_Date, Close_Date, First_Bill_Date, Forecast_Category,Forecast_pd,
CONCAT(Forecast_pd,'_') AS Forecast_pd2, SUM(Annual_Amt) AS Annual_Amt, SUM(Value) AS Value
FROM CREATE_UID
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26),
PIVOT_TABLE AS (
select *
FROM CTE3
PIVOT (SUM(Value) FOR Forecast_pd IN ('Jan_24','Feb_24','Mar_24','Apr_24','May_24','Jun_24','Jul_24','Aug_24','Sep_24','Oct_24','Nov_24','Dec_24',
'Jan_25','Feb_25','Mar_25','Apr_25','May_25','Jun_25','Jul_25','Aug_25','Sep_25','Oct_25','Nov_25','Dec_25')))
SELECT Region,
Cluster,
Country,
BU,
Product,
StoSer,
Segment,
RNR,
Phase,
Stage,
UID,
Opp_ID,
Opportunity_Name,
Deal_Size,
Age,
Account_Name,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
Probability,
Created_Date,
Close_Date,
First_Bill_Date,
Forecast_Category,
SUM(Annual_Amt) AS Annual_Amt,
SUM(Jan_24) AS SF1_Jan,
SUM(Feb_24) AS SF1_Feb,
SUM(Mar_24) AS SF1_Mar,
SUM(Apr_24) AS SF1_Apr,
SUM(May_24) AS SF1_May,
SUM(Jun_24) AS SF1_Jun,
SUM(Jul_24) AS SF1_Jul,
SUM(Aug_24) AS SF1_Aug,
SUM(Sep_24) AS SF1_Sep,
SUM(Oct_24) AS SF1_Oct,
SUM(Nov_24) AS SF1_Nov,
SUM(Dec_24) AS SF1_Dec,
SUM(IFNULL(Jan_24,0)+IFNULL(Feb_24,0)+IFNULL(Mar_24,0)+IFNULL(Apr_24,0)+IFNULL(May_24,0)+IFNULL(Jun_24,0)+IFNULL(Jul_24,0)+IFNULL(Aug_24,0)+IFNULL(Sep_24,0)+IFNULL(Oct_24,0)+IFNULL(Nov_24,0)+IFNULL(Dec_24,0)) AS SF1_FY,
SUM(Jan_25) AS SF2_Jan,
SUM(Feb_25) AS SF2_Feb,
SUM(Mar_25) AS SF2_Mar,
SUM(Apr_25) AS SF2_Apr,
SUM(May_25) AS SF2_May,
SUM(Jun_25) AS SF2_Jun,
SUM(Jul_25) AS SF2_Jul,
SUM(Aug_25) AS SF2_Aug,
SUM(Sep_25) AS SF2_Sep,
SUM(Oct_25) AS SF2_Oct,
SUM(Nov_25) AS SF2_Nov,
SUM(Dec_25) AS SF2_Dec,
SUM(IFNULL(Jan_25,0)+IFNULL(Feb_25,0)+IFNULL(Mar_25,0)+IFNULL(Apr_25,0)+IFNULL(May_25,0)+IFNULL(Jun_25,0)+IFNULL(Jul_25,0)+IFNULL(Aug_25,0)+IFNULL(Sep_25,0)+IFNULL(Oct_25,0)+IFNULL(Nov_25,0)+IFNULL(Dec_25,0)) AS SF2_FY
FROM PIVOT_TABLE
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25)
''')

In [ ]:
_ = query_bq('''
CREATE OR REPLACE TABLE irm-fin-acct-dp-prod.IYR_Process.B2B_Clean_Digital AS (
WITH BU_Lookup AS(
SELECT DISTINCT Product,BU
FROM irm-fin-acct-dp-prod.IYR_Process.CC_Lookups),

ProductCleanse AS (SELECT DISTINCT ProdF,ProdR,ProdRDigi
FROM irm-fin-acct-dp-prod.IYR_Process.CC_Cleanse WHERE ProdF <> ''),

DigitalLookup AS (SELECT DISTINCT Prod_ClassLineName,Prod_Digital_New,Prod_Digital_Hist
FROM irm-fin-acct-dp-prod.IYR_Process.CC_Lookups WHERE Prod_ClassLineName <> ''),

FILTER_B2B AS (
SELECT *
FROM irm-fin-acct-dp-prod.IYR_Process.B2B_In_Snap
WHERE Product_Class <> 'IMDC'
AND  Product_Class <> 'ITRenew'
AND Product_Class <> 'ALM'
AND Forecast_Category <> 'Omitted'
AND Total_Annual_Amount <> 0.0
AND (Stage <> '1 - Qualification' OR (Cluster = 'EMEA: MENATSA' AND Product_Class <> 'MAS'))
AND (
  (Forecast_Revenue_Date >= CAST('2024-01-01' AS DATE) AND Forecast_Revenue_Date < CAST('2026-01-01' AS DATE))
  OR
  (Close_Date >= CAST('2023-01-01' AS DATE) AND Close_Date < CAST('2026-01-01' AS DATE))
  OR
  (Opportunity_ID IN (SELECT OppID FROM irm-fin-acct-dp-prod.IYR_Process.CC_2022OppIDS))
    )
),

CTE AS (SELECT Opportunity_ID AS Opp_ID,
Opportunity_Name,
Account_Name,
CASE WHEN Product_Class = 'IMES' THEN 'IMES' ELSE Region END AS Region,
Country,
CASE WHEN Cluster = 'Canada' THEN 'NA: Canada' WHEN LEFT(Cluster,2) = 'US' THEN REPLACE(Cluster,'US:','NA:') ELSE Cluster END AS Cluster,
Prod_Digital_New AS Product,
CASE WHEN Commission_Category = 'M1' THEN 'Storage' ELSE 'Service' END AS StoSer,
CASE WHEN Segment = 'Global Industries' THEN 'GI' ELSE 'Region' END AS Segment,
CASE WHEN Revenue_Type = 'Non Recurring' THEN 'Non-recurring' ELSE Revenue_Type END AS RNR,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
CASE WHEN Product_Revenue_Band IN ("$80k - $250k" , "$25k - $80k") THEN "$25k - $250k" ELSE Product_Revenue_Band END AS Deal_Size,
CASE
WHEN SAFE_CAST(LEFT(Stage,1) AS INTEGER)<= 4  OR (Stage = 'Pipeline') THEN 'Pipeline'
WHEN (SAFE_CAST(LEFT(Stage,1) AS INTEGER)>= 5  AND SAFE_CAST(LEFT(Stage,1) AS INTEGER)<= 7) OR (Stage = 'Bookings') THEN 'Bookings'
ELSE NULL END AS Phase,
Stage,
Forecast_Category,
Forecast_Type,
Probability,
Age,
Created_Date,
Close_Date,
CASE WHEN REGION = 'NA' AND Close_Date IS NOT NULL AND First_Bill_Date IS NULL THEN DATE_ADD(Close_Date, INTERVAL 90 DAY) ELSE First_Bill_Date END AS First_Bill_Date,
Forecast_First_Bill_Date,
Forecasted_Billings,
Booking_Forecast,
Total_Annual_Amount AS Annual_Amt,
Forecast_Total,
Forecast_Revenue_Date,
IFNULL(Pipeline_Forecast,0) AS Pipeline_Forecast,
IFNULL(Sales_Forecast,0) AS Sales_Forecast,
IFNULL(Ops_Forecast,0) AS Ops_Forecast,
IFNULL(Booked_Assumptions,0) AS Booked_Assumptions,
IFNULL(Actual_Billings,0) AS Actual_Billings,
CURRENT_TIMESTAMP() AS Timestamp,
'B2B' AS Data_Source,
IFNULL(Pipeline_Forecast,0)+IFNULL(Sales_Forecast,0)+IFNULL(Ops_Forecast,0)+IFNULL(Booked_Assumptions,0)+IFNULL(Actual_Billings,0) AS Value,
CAST(FORMAT_DATETIME('%b_%y',Forecast_Revenue_Date) AS STRING) AS Forecast_pd,
BU_Lookup.BU AS BU
FROM FILTER_B2B
LEFT JOIN ProductCleanse ON Product_Class=ProductCleanse.ProdF
LEFT JOIN BU_Lookup ON ProductCleanse.ProdR = BU_Lookup.Product
LEFT JOIN DigitalLookup ON CONCAT(Product_Class,Product_Line,Product_Name)= DigitalLookup.Prod_ClassLineName),
CREATE_UID AS(
SELECT *,CONCAT(Opp_ID,'-',Country,'-',Product,'-',StoSer,'-',RNR,'-',Deal_Size) AS UID
FROM CTE
WHERE BU = 'Digital'),

CTE3 AS (
SELECT Region, Cluster, Country,    BU, Product,    StoSer, Segment,RNR, Phase,Stage, UID, Opp_ID, Opportunity_Name, Deal_Size, Age, Account_Name, GI_Vertical,  Deal_Type, Salesforce_URL, Opportunity_Owner, Probability, Created_Date, Close_Date, First_Bill_Date, Forecast_Category,Forecast_pd,
CONCAT(Forecast_pd,'_') AS Forecast_pd2, SUM(Annual_Amt) AS Annual_Amt, SUM(Value) AS Value
FROM CREATE_UID
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26),
PIVOT_TABLE AS (
select *
FROM CTE3
PIVOT (SUM(Value) FOR Forecast_pd IN ('Jan_24','Feb_24','Mar_24','Apr_24','May_24','Jun_24','Jul_24','Aug_24','Sep_24','Oct_24','Nov_24','Dec_24',
'Jan_25','Feb_25','Mar_25','Apr_25','May_25','Jun_25','Jul_25','Aug_25','Sep_25','Oct_25','Nov_25','Dec_25')))
SELECT Region,
Cluster,
Country,
BU,
Product,
StoSer,
Segment,
RNR,
Phase,
Stage,
UID,
Opp_ID,
Opportunity_Name,
Deal_Size,
Age,
Account_Name,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
Probability,
Created_Date,
Close_Date,
First_Bill_Date,
Forecast_Category,
SUM(Annual_Amt) AS Annual_Amt,
SUM(Jan_24) AS SF1_Jan,
SUM(Feb_24) AS SF1_Feb,
SUM(Mar_24) AS SF1_Mar,
SUM(Apr_24) AS SF1_Apr,
SUM(May_24) AS SF1_May,
SUM(Jun_24) AS SF1_Jun,
SUM(Jul_24) AS SF1_Jul,
SUM(Aug_24) AS SF1_Aug,
SUM(Sep_24) AS SF1_Sep,
SUM(Oct_24) AS SF1_Oct,
SUM(Nov_24) AS SF1_Nov,
SUM(Dec_24) AS SF1_Dec,
SUM(IFNULL(Jan_24,0)+IFNULL(Feb_24,0)+IFNULL(Mar_24,0)+IFNULL(Apr_24,0)+IFNULL(May_24,0)+IFNULL(Jun_24,0)+IFNULL(Jul_24,0)+IFNULL(Aug_24,0)+IFNULL(Sep_24,0)+IFNULL(Oct_24,0)+IFNULL(Nov_24,0)+IFNULL(Dec_24,0)) AS SF1_FY,
SUM(Jan_25) AS SF2_Jan,
SUM(Feb_25) AS SF2_Feb,
SUM(Mar_25) AS SF2_Mar,
SUM(Apr_25) AS SF2_Apr,
SUM(May_25) AS SF2_May,
SUM(Jun_25) AS SF2_Jun,
SUM(Jul_25) AS SF2_Jul,
SUM(Aug_25) AS SF2_Aug,
SUM(Sep_25) AS SF2_Sep,
SUM(Oct_25) AS SF2_Oct,
SUM(Nov_25) AS SF2_Nov,
SUM(Dec_25) AS SF2_Dec,
SUM(IFNULL(Jan_25,0)+IFNULL(Feb_25,0)+IFNULL(Mar_25,0)+IFNULL(Apr_25,0)+IFNULL(May_25,0)+IFNULL(Jun_25,0)+IFNULL(Jul_25,0)+IFNULL(Aug_25,0)+IFNULL(Sep_25,0)+IFNULL(Oct_25,0)+IFNULL(Nov_25,0)+IFNULL(Dec_25,0)) AS SF2_FY
FROM PIVOT_TABLE
GROUP BY 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
)

''')

#### Union_Ins

In [ ]:
_ = query_bq('''
CREATE OR REPLACE TABLE irm-fin-acct-dp-prod.IYR_Process.Union_Ins AS(
SELECT
Time_Stamp,
DATE_TRUNC(CURRENT_DATE(),WEEK(MONDAY)) AS Wk_Ending,
'B2B-E2E-Manual' as DataSource,
Region,
Cluster,
a.Country AS Country,
BU,
Product,
CAST(NULL AS STRING) AS SubProduct,
StoSer,
Segment,
RNR,
Phase,
Stage,
UID,
Opp_ID,
Opportunity_Name,
Deal_Size,
Age,
Account_Name,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
Probability,
Created_Date,
Close_Date,
First_Bill_Date,
Forecast_Category,
Annual_Amt,
EB AS MEB,
--Source,
SF1_Jan*(1-IFNULL(Tax.Tax,0)) AS SF1_Jan,
SF1_Feb*(1-IFNULL(Tax.Tax,0)) AS SF1_Feb,
SF1_Mar*(1-IFNULL(Tax.Tax,0)) AS SF1_Mar,
SF1_Apr*(1-IFNULL(Tax.Tax,0)) AS SF1_Apr,
SF1_May*(1-IFNULL(Tax.Tax,0)) AS SF1_May,
SF1_Jun*(1-IFNULL(Tax.Tax,0)) AS SF1_Jun,
SF1_Jul*(1-IFNULL(Tax.Tax,0)) AS SF1_Jul,
SF1_Aug*(1-IFNULL(Tax.Tax,0)) AS SF1_Aug,
SF1_Sep*(1-IFNULL(Tax.Tax,0)) AS SF1_Sep,
SF1_Oct*(1-IFNULL(Tax.Tax,0)) AS SF1_Oct,
SF1_Nov*(1-IFNULL(Tax.Tax,0)) AS SF1_Nov,
SF1_Dec*(1-IFNULL(Tax.Tax,0)) AS SF1_Dec,
SF1_FY*(1-IFNULL(Tax.Tax,0)) AS SF1_FY,
SF2_Jan*(1-IFNULL(Tax.Tax,0)) AS SF2_Jan,
SF2_Feb*(1-IFNULL(Tax.Tax,0)) AS SF2_Feb,
SF2_Mar*(1-IFNULL(Tax.Tax,0)) AS SF2_Mar,
SF2_Apr*(1-IFNULL(Tax.Tax,0)) AS SF2_Apr,
SF2_May*(1-IFNULL(Tax.Tax,0)) AS SF2_May,
SF2_Jun*(1-IFNULL(Tax.Tax,0)) AS SF2_Jun,
SF2_Jul*(1-IFNULL(Tax.Tax,0)) AS SF2_Jul,
SF2_Aug*(1-IFNULL(Tax.Tax,0)) AS SF2_Aug,
SF2_Sep*(1-IFNULL(Tax.Tax,0)) AS SF2_Sep,
SF2_Oct*(1-IFNULL(Tax.Tax,0)) AS SF2_Oct,
SF2_Nov*(1-IFNULL(Tax.Tax,0)) AS SF2_Nov,
SF2_Dec*(1-IFNULL(Tax.Tax,0)) AS SF2_Dec,
SF2_FY*(1-IFNULL(Tax.Tax,0)) AS SF2_FY,
EE1_Jan*(1-IFNULL(Tax.Tax,0)) AS EE1_Jan,
EE1_Feb*(1-IFNULL(Tax.Tax,0)) AS EE1_Feb,
EE1_Mar*(1-IFNULL(Tax.Tax,0)) AS EE1_Mar,
EE1_Apr*(1-IFNULL(Tax.Tax,0)) AS EE1_Apr,
EE1_May*(1-IFNULL(Tax.Tax,0)) AS EE1_May,
EE1_Jun*(1-IFNULL(Tax.Tax,0)) AS EE1_Jun,
EE1_Jul*(1-IFNULL(Tax.Tax,0)) AS EE1_Jul,
EE1_Aug*(1-IFNULL(Tax.Tax,0)) AS EE1_Aug,
EE1_Sep*(1-IFNULL(Tax.Tax,0)) AS EE1_Sep,
EE1_Oct*(1-IFNULL(Tax.Tax,0)) AS EE1_Oct,
EE1_Nov*(1-IFNULL(Tax.Tax,0)) AS EE1_Nov,
EE1_Dec*(1-IFNULL(Tax.Tax,0)) AS EE1_Dec,
EE1_FY*(1-IFNULL(Tax.Tax,0)) AS EE1_FY,
EE2_Jan*(1-IFNULL(Tax.Tax,0)) AS EE2_Jan,
EE2_Feb*(1-IFNULL(Tax.Tax,0)) AS EE2_Feb,
EE2_Mar*(1-IFNULL(Tax.Tax,0)) AS EE2_Mar,
EE2_Apr*(1-IFNULL(Tax.Tax,0)) AS EE2_Apr,
EE2_May*(1-IFNULL(Tax.Tax,0)) AS EE2_May,
EE2_Jun*(1-IFNULL(Tax.Tax,0)) AS EE2_Jun,
EE2_Jul*(1-IFNULL(Tax.Tax,0)) AS EE2_Jul,
EE2_Aug*(1-IFNULL(Tax.Tax,0)) AS EE2_Aug,
EE2_Sep*(1-IFNULL(Tax.Tax,0)) AS EE2_Sep,
EE2_Oct*(1-IFNULL(Tax.Tax,0)) AS EE2_Oct,
EE2_Nov*(1-IFNULL(Tax.Tax,0)) AS EE2_Nov,
EE2_Dec*(1-IFNULL(Tax.Tax,0)) AS EE2_Dec,
EE2_FY*(1-IFNULL(Tax.Tax,0)) AS EE2_FY,
--Comments,
Join_Flag   ,
CAST(NULL AS STRING) AS E2E_Match,--E2E
CAST(NULL AS STRING) AS Memo,--Product_Subcategory E2E,Pr Fcst,RO
CAST(NULL AS STRING) AS Site, --E2E
CAST(NULL AS STRING) AS Customer_ID, --E2E
CAST(NULL AS STRING) AS Project,--E2E
CAST(NULL AS STRING) AS Project_Type,--E2E
NULL AS Book_to_Bill,--Ass
NULL AS Estimated,--Ass
NULL AS No_of_Billing,--Ass
CAST(NULL AS FLOAT64) AS M1,--Ass
CAST(NULL AS FLOAT64) AS M2,--Ass
CAST(NULL AS FLOAT64) AS M3,--Ass
CAST(NULL AS FLOAT64) AS M4,--Ass
CAST(NULL AS FLOAT64) AS M5,--Ass
CAST(NULL AS FLOAT64) AS M6,--Ass
CAST(NULL AS FLOAT64) AS M7,--Ass
CAST(NULL AS FLOAT64) AS M8,--Ass
CAST(NULL AS FLOAT64) AS M9,--Ass
CAST(NULL AS FLOAT64) AS M10,--Ass
CAST(NULL AS FLOAT64) AS M11,--Ass
CAST(NULL AS FLOAT64) AS M12--Ass
FROM irm-fin-acct-dp-prod.IYR_Process.B2B_E2E_Join  a LEFT JOIN irm-fin-acct-dp-prod.IYR.Tax AS Tax ON a.Country = Tax.Country
-------------------------------------------------------------------------------------------------------------------------------------
UNION ALL
SELECT
Time_Stamp,
DATE_TRUNC(CURRENT_DATE(),WEEK(MONDAY)) AS Wk_Ending,
'E2E Detail' as DataSource,
Region,
Cluster,
a.Country AS Country,
BU,
Product,
CAST(NULL AS STRING) AS SubProduct,
StoSer,
Segment,
RNR,
CAST(NULL AS STRING) AS Phase,
CAST(NULL AS STRING) AS Stage,
UID ,
Opp_ID  ,
CAST(NULL AS STRING) AS Opportunity_Name,
Deal_Size,
NULL AS Age,
CAST(NULL AS STRING) AS Account_Name,
CAST(NULL AS STRING) AS GI_Vertical,
NULL AS Deal_Type,
CAST(NULL AS STRING) AS Salesforce_URL,
CAST(NULL AS STRING) AS Opportunity_Owner,
NULL Probability,
NULL Created_Date,
NULL Close_Date,
NULL First_Bill_Date,
NULL Forecast_Category,
NULL Annual_Amt,
NULL AS MEB,
--Source,
NULL AS SF1_Jan,
NULL AS SF1_Feb,
NULL AS SF1_Mar,
NULL AS SF1_Apr,
NULL AS SF1_May,
NULL AS SF1_Jun,
NULL AS SF1_Jul,
NULL AS SF1_Aug,
NULL AS SF1_Sep,
NULL AS SF1_Oct,
NULL AS SF1_Nov,
NULL AS SF1_Dec,
NULL AS SF1_FY,
NULL AS SF2_Jan,
NULL AS SF2_Feb,
NULL AS SF2_Mar,
NULL AS SF2_Apr,
NULL AS SF2_May,
NULL AS SF2_Jun,
NULL AS SF2_Jul,
NULL AS SF2_Aug,
NULL AS SF2_Sep,
NULL AS SF2_Oct,
NULL AS SF2_Nov,
NULL AS SF2_Dec,
NULL AS SF2_FY,
EE1_Jan*(1-IFNULL(Tax.Tax,0)) AS EE1_Jan,
EE1_Feb*(1-IFNULL(Tax.Tax,0)) AS EE1_Feb,
EE1_Mar*(1-IFNULL(Tax.Tax,0)) AS EE1_Mar,
EE1_Apr*(1-IFNULL(Tax.Tax,0)) AS EE1_Apr,
EE1_May*(1-IFNULL(Tax.Tax,0)) AS EE1_May,
EE1_Jun*(1-IFNULL(Tax.Tax,0)) AS EE1_Jun,
EE1_Jul*(1-IFNULL(Tax.Tax,0)) AS EE1_Jul,
EE1_Aug*(1-IFNULL(Tax.Tax,0)) AS EE1_Aug,
EE1_Sep*(1-IFNULL(Tax.Tax,0)) AS EE1_Sep,
EE1_Oct*(1-IFNULL(Tax.Tax,0)) AS EE1_Oct,
EE1_Nov*(1-IFNULL(Tax.Tax,0)) AS EE1_Nov,
EE1_Dec*(1-IFNULL(Tax.Tax,0)) AS EE1_Dec,
EE1_FY*(1-IFNULL(Tax.Tax,0)) AS EE1_FY,
EE2_Jan*(1-IFNULL(Tax.Tax,0)) AS EE2_Jan,
EE2_Feb*(1-IFNULL(Tax.Tax,0)) AS EE2_Feb,
EE2_Mar*(1-IFNULL(Tax.Tax,0)) AS EE2_Mar,
EE2_Apr*(1-IFNULL(Tax.Tax,0)) AS EE2_Apr,
EE2_May*(1-IFNULL(Tax.Tax,0)) AS EE2_May,
EE2_Jun*(1-IFNULL(Tax.Tax,0)) AS EE2_Jun,
EE2_Jul*(1-IFNULL(Tax.Tax,0)) AS EE2_Jul,
EE2_Aug*(1-IFNULL(Tax.Tax,0)) AS EE2_Aug,
EE2_Sep*(1-IFNULL(Tax.Tax,0)) AS EE2_Sep,
EE2_Oct*(1-IFNULL(Tax.Tax,0)) AS EE2_Oct,
EE2_Nov*(1-IFNULL(Tax.Tax,0)) AS EE2_Nov,
EE2_Dec*(1-IFNULL(Tax.Tax,0)) AS EE2_Dec,
EE2_FY*(1-IFNULL(Tax.Tax,0)) AS EE2_FY,
--Comments,
Join_Flag   ,
E2E_Match,--E2E
Product_SubCategory AS Memo,--Product_Subcategory E2E,Pr Fcst,RO
Site, --E2E
Customer_ID, --E2E
Project,--E2E
Project_Type,--E2E
NULL AS Book_to_Bill,--Ass
NULL AS Estimated,--Ass
NULL AS No_of_Billing,--Ass
CAST(NULL AS FLOAT64) AS M1,--Ass
CAST(NULL AS FLOAT64) AS M2,--Ass
CAST(NULL AS FLOAT64) AS M3,--Ass
CAST(NULL AS FLOAT64) AS M4,--Ass
CAST(NULL AS FLOAT64) AS M5,--Ass
CAST(NULL AS FLOAT64) AS M6,--Ass
CAST(NULL AS FLOAT64) AS M7,--Ass
CAST(NULL AS FLOAT64) AS M8,--Ass
CAST(NULL AS FLOAT64) AS M9,--Ass
CAST(NULL AS FLOAT64) AS M10,--Ass
CAST(NULL AS FLOAT64) AS M11,--Ass
CAST(NULL AS FLOAT64) AS M12--Ass
FROM (irm-fin-acct-dp-prod.IYR_Process.E2E_Clean  a LEFT JOIN irm-fin-acct-dp-prod.IYR.Tax AS Tax ON a.Country = Tax.Country)
----------------------------------------------------------------------------------------------------------------------------------------
UNION ALL
SELECT
Time_Stamp,
DATE_TRUNC(CURRENT_DATE(),WEEK(MONDAY)) AS Wk_Ending,
'B2B Assumptions' as DataSource,
Region,
Cluster,
Country,
BU,
Product AS Product,
CAST(NULL AS STRING) AS SubProduct,
StoSer,
Segment,
RNR,
NULL Phase,
NULL Stage,
NULL UID,
NULL Opp_ID,
NULL Opportunity_Name,
RevBand AS Deal_Size,
NULL Age,
NULL Account_Name,
NULL GI_Vertical,
NULL Deal_Type,
NULL Salesforce_URL,
NULL Opportunity_Owner,
NULL Probability,
NULl Created_Date,
NULL Close_Date,
NULL First_Bill_Date,
NULL Forecast_Category,
NULL Annual_Amt,
NULL MEB,
--Source,
NULL AS SF1_Jan,
NULL AS SF1_Feb,
NULL AS SF1_Mar,
NULL AS SF1_Apr,
NULL AS SF1_May,
NULL AS SF1_Jun,
NULL AS SF1_Jul,
NULL AS SF1_Aug,
NULL AS SF1_Sep,
NULL AS SF1_Oct,
NULL AS SF1_Nov,
NULL AS SF1_Dec,
NULL AS SF1_FY,
NULL AS SF2_Jan,
NULL AS SF2_Feb,
NULL AS SF2_Mar,
NULL AS SF2_Apr,
NULL AS SF2_May,
NULL AS SF2_Jun,
NULL AS SF2_Jul,
NULL AS SF2_Aug,
NULL AS SF2_Sep,
NULL AS SF2_Oct,
NULL AS SF2_Nov,
NULL AS SF2_Dec,
NULL AS SF2_FY,
NULL AS EE1_Jan,
NULL AS EE1_Feb,
NULL AS EE1_Mar,
NULL AS EE1_Apr,
NULL AS EE1_May,
NULL AS EE1_Jun,
NULL AS EE1_Jul,
NULL AS EE1_Aug,
NULL AS EE1_Sep,
NULL AS EE1_Oct,
NULL AS EE1_Nov,
NULL AS EE1_Dec,
NULL AS EE1_FY,
NULL AS EE2_Jan,
NULL AS EE2_Feb,
NULL AS EE2_Mar,
NULL AS EE2_Apr,
NULL AS EE2_May,
NULL AS EE2_Jun,
NULL AS EE2_Jul,
NULL AS EE2_Aug,
NULL AS EE2_Sep,
NULL AS EE2_Oct,
NULL AS EE2_Nov,
NULL AS EE2_Dec,
NULL AS EE2_FY,
--Comments,
NULL Join_Flag  ,
CAST(NULL AS STRING) AS ASE2E_Match,--E2E
CAST(NULL AS STRING) AS Memo,--Product_Subcategory E2E,Pr Fcst,RO
CAST(NULL AS STRING) AS Site, --E2E
CAST(NULL AS STRING) AS Customer_ID, --E2E
CAST(NULL AS STRING) AS Project,--E2E
CAST(NULL AS STRING) AS Project_Type,--E2E
Book_to_Bill_Ratio AS Book_to_Bill,--Ass
Estimated_Close_to_First_Bill_Date AS Estimated,--Ass
No_of_Billing_Months AS No_of_Billing,--Ass
Month_1 AS M1,--Ass
Month_2 AS M2,--Ass
Month_3 AS M3,--Ass
Month_4 AS M4,--Ass
Month_5 AS M5,--Ass
Month_6 AS M6,--Ass
Month_7 AS M7,--Ass
Month_8 AS M8,--Ass
Month_9 AS M9,--Ass
Month_10 AS M10,--Ass
Month_11 AS M11,--Ass
Month_12 AS M12--Ass
FROM irm-fin-acct-dp-prod.IYR_Process.B2B_Assumptions_Clean
UNION ALL

SELECT
Time_Stamp,
DATE_TRUNC(CURRENT_DATE(),WEEK(MONDAY)) AS Wk_Ending,
'IYR Budget' as DataSource,
Region,
Cluster,
Country,
BU,
Product AS Product,
CAST(NULL AS STRING) AS SubProduct,
StoSer,
Segment,
RNR,
NULL Phase,
NULL Stage,
NULL UID,
NULL Opp_ID,
NULL Opportunity_Name,
NULL Deal_Size,
NULL Age,
NULL Account_Name,
NULL GI_Vertical,
NULL Deal_Type,
NULL Salesforce_URL,
NULL Opportunity_Owner,
NULL Probability,
NULl Created_Date,
NULL Close_Date,
NULL First_Bill_Date,
NULL Forecast_Category,
NULL Annual_Amt,
NULL MEB,
--Source,
Jan AS SF1_Jan,
Feb AS SF1_Feb,
Mar AS SF1_Mar,
Apr AS SF1_Apr,
May AS SF1_May,
Jun AS SF1_Jun,
Jul AS SF1_Jul,
Aug AS SF1_Aug,
Sep AS SF1_Sep,
Oct AS SF1_Oct,
Nov AS SF1_Nov,
Dec AS SF1_Dec,
FY AS SF1_FY,
NULL AS SF2_Jan,
NULL AS SF2_Feb,
NULL AS SF2_Mar,
NULL AS SF2_Apr,
NULL AS SF2_May,
NULL AS SF2_Jun,
NULL AS SF2_Jul,
NULL AS SF2_Aug,
NULL AS SF2_Sep,
NULL AS SF2_Oct,
NULL AS SF2_Nov,
NULL AS SF2_Dec,
NULL AS SF2_FY,
NULL AS EE1_Jan,
NULL AS EE1_Feb,
NULL AS EE1_Mar,
NULL AS EE1_Apr,
NULL AS EE1_May,
NULL AS EE1_Jun,
NULL AS EE1_Jul,
NULL AS EE1_Aug,
NULL AS EE1_Sep,
NULL AS EE1_Oct,
NULL AS EE1_Nov,
NULL AS EE1_Dec,
NULL AS EE1_FY,
NULL AS EE2_Jan,
NULL AS EE2_Feb,
NULL AS EE2_Mar,
NULL AS EE2_Apr,
NULL AS EE2_May,
NULL AS EE2_Jun,
NULL AS EE2_Jul,
NULL AS EE2_Aug,
NULL AS EE2_Sep,
NULL AS EE2_Oct,
NULL AS EE2_Nov,
NULL AS EE2_Dec,
NULL AS EE2_FY,
--Comments,
NULL Join_Flag  ,
CAST(NULL AS STRING) AS E2E_Match,--E2E
ProductDetail AS Memo,--Product_Subcategory E2E,Pr Fcst,RO
CAST(NULL AS STRING) AS Site, --E2E
CAST(NULL AS STRING) AS Customer_ID, --E2E
CAST(NULL AS STRING) AS Project,--E2E
CAST(NULL AS STRING) AS Project_Type,--E2E
NULL AS Book_to_Bill,--Ass
NULL AS Estimated,--Ass
NULL AS No_of_Billing,--Ass
NULL AS M1,--Ass
NULL AS M2,--Ass
NULL AS M3,--Ass
NULL AS M4,--Ass
NULL AS M5,--Ass
NULL AS M6,--Ass
NULL AS M7,--Ass
NULL AS M8,--Ass
NULL AS M9,--Ass
NULL AS M10,--Ass
NULL AS M11,--Ass
NULL AS M12--Ass
FROM irm-fin-acct-dp-prod.IYR_Process.IYRBudData_Clean
UNION ALL
SELECT
Time_Stamp,
DATE_TRUNC(CURRENT_DATE(),WEEK(MONDAY)) AS Wk_Ending,
'Pr Fcst' as DataSource,
Region,
Cluster,
Country,
BU,
Product,
CAST(NULL AS STRING) AS SubProduct,
StoSer,
Segment,
RNR,
NULL Phase  ,
NULL Stage  ,
NULL UID    ,
NULL Opp_ID ,
NULL Opportunity_Name,
NULL Deal_Size,
NULL Age,
NULL Account_Name,
NULL GI_Vertical,
NULL Deal_Type,
NULL Salesforce_URL,
NULL Opportunity_Owner,
NULL Probability,
NULL Created_Date,
NULL Close_Date,
NULL First_Bill_Date,
NULL Forecast_Category,
NULL Annual_Amt,
NULL AS MEB,
--Source,
Jan AS SF1_Jan,
Feb AS SF1_Feb,
Mar AS SF1_Mar,
Apr AS SF1_Apr,
May AS SF1_May,
Jun AS SF1_Jun,
Jul AS SF1_Jul,
Aug AS SF1_Aug,
Sep AS SF1_Sep,
Oct AS SF1_Oct,
Nov AS SF1_Nov,
Dec AS SF1_Dec,
FY AS SF1_FY,
NULL AS SF2_Jan,
NULL AS SF2_Feb,
NULL AS SF2_Mar,
NULL AS SF2_Apr,
NULL AS SF2_May,
NULL AS SF2_Jun,
NULL AS SF2_Jul,
NULL AS SF2_Aug,
NULL AS SF2_Sep,
NULL AS SF2_Oct,
NULL AS SF2_Nov,
NULL AS SF2_Dec,
NULL AS SF2_FY,
NULL AS EE1_Jan,
NULL AS EE1_Feb,
NULL AS EE1_Mar,
NULL AS EE1_Apr,
NULL AS EE1_May,
NULL AS EE1_Jun,
NULL AS EE1_Jul,
NULL AS EE1_Aug,
NULL AS EE1_Sep,
NULL AS EE1_Oct,
NULL AS EE1_Nov,
NULL AS EE1_Dec,
NULL AS EE1_FY,
NULL AS EE2_Jan,
NULL AS EE2_Feb,
NULL AS EE2_Mar,
NULL AS EE2_Apr,
NULL AS EE2_May,
NULL AS EE2_Jun,
NULL AS EE2_Jul,
NULL AS EE2_Aug,
NULL AS EE2_Sep,
NULL AS EE2_Oct,
NULL AS EE2_Nov,
NULL AS EE2_Dec,
NULL AS EE2_FY,
--Comments,
NULL Join_Flag  ,
CAST(NULL AS STRING) AS ASE2E_Match,--E2E
ProductDetail AS Memo,--Product_Subcategory E2E,Pr Fcst,RO
CAST(NULL AS STRING) AS Site, --E2E
CAST(NULL AS STRING) AS Customer_ID, --E2E
CAST(NULL AS STRING) AS Project,--E2E
CAST(NULL AS STRING) AS Project_Type,--E2E
NULL AS Book_to_Bill,--Ass
NULL AS Estimated,--Ass
NULL AS No_of_Billing,--Ass
CAST(NULL AS FLOAT64) AS M1,--Ass
CAST(NULL AS FLOAT64) AS M2,--Ass
CAST(NULL AS FLOAT64) AS M3,--Ass
CAST(NULL AS FLOAT64) AS M4,--Ass
CAST(NULL AS FLOAT64) AS M5,--Ass
CAST(NULL AS FLOAT64) AS M6,--Ass
CAST(NULL AS FLOAT64) AS M7,--Ass
CAST(NULL AS FLOAT64) AS M8,--Ass
CAST(NULL AS FLOAT64) AS M9,--Ass
CAST(NULL AS FLOAT64) AS M10,--Ass
CAST(NULL AS FLOAT64) AS M11,--Ass
CAST(NULL AS FLOAT64) AS M12--Ass
FROM irm-fin-acct-dp-prod.IYR_Process.Fcst_Prior_Clean
UNION ALL
SELECT
Time_Stamp,
DATE_TRUNC(CURRENT_DATE(),WEEK(MONDAY)) AS Wk_Ending,
'R&O Tracker' as DataSource,
Region,
Cluster,
Country AS Country,
BU,
Product,
CAST(NULL AS STRING) AS SubProduct,
Function AS StoSer,
Segment,
RNR,
NULL Phase,
NULL Stage,
NULL UID,
NULL Opp_ID,
NULL Opportunity_Name,
NULL Deal_Size,
NULL Age,
NULL Account_Name,
NULL GI_Vertical,
NULL Deal_Type,
NULL Salesforce_URL,
NULL Opportunity_Owner,
NULL Probability,
NULL Created_Date,
NULL Close_Date,
NULL First_Bill_Date,
NULL Forecast_Category,
NULL Annual_Amt,
NULL AS MEB,
--Source,
Jan_2024_Revenue AS SF1_Jan,
Feb_2024_Revenue AS SF1_Feb,
Mar_2024_Revenue AS SF1_Mar,
Apr_2024_Revenue AS SF1_Apr,
May_2024_Revenue AS SF1_May,
Jun_2024_Revenue AS SF1_Jun,
Jul_2024_Revenue AS SF1_Jul,
Aug_2024_Revenue AS SF1_Aug,
Sep_2024_Revenue AS SF1_Sep,
Oct_2024_Revenue AS SF1_Oct,
Nov_2024_Revenue AS SF1_Nov,
Dec_2024_Revenue AS SF1_Dec,
FY_2024 AS SF1_FY,
Jan_2025_Revenue AS SF2_Jan,
Feb_2025_Revenue AS SF2_Feb,
Mar_2025_Revenue AS SF2_Mar,
Apr_2025_Revenue AS SF2_Apr,
May_2025_Revenue AS SF2_May,
Jun_2025_Revenue AS SF2_Jun,
Jul_2025_Revenue AS SF2_Jul,
Aug_2025_Revenue AS SF2_Aug,
Sep_2025_Revenue AS SF2_Sep,
Oct_2025_Revenue AS SF2_Oct,
Nov_2025_Revenue AS SF2_Nov,
Dec_2025_Revenue AS SF2_Dec,
FY_2025 AS SF2_FY,
NULL AS EE1_Jan,
NULL AS EE1_Feb,
NULL AS EE1_Mar,
NULL AS EE1_Apr,
NULL AS EE1_May,
NULL AS EE1_Jun,
NULL AS EE1_Jul,
NULL AS EE1_Aug,
NULL AS EE1_Sep,
NULL AS EE1_Oct,
NULL AS EE1_Nov,
NULL AS EE1_Dec,
NULL AS EE1_FY,
NULL AS EE2_Jan,
NULL AS EE2_Feb,
NULL AS EE2_Mar,
NULL AS EE2_Apr,
NULL AS EE2_May,
NULL AS EE2_Jun,
NULL AS EE2_Jul,
NULL AS EE2_Aug,
NULL AS EE2_Sep,
NULL AS EE2_Oct,
NULL AS EE2_Nov,
NULL AS EE2_Dec,
NULL AS EE2_FY,
--Comments,
NULL Join_Flag  ,
CAST(NULL AS STRING) AS E2E_Match,--E2E
Product_SubCategory AS Memo,--Product_Subcategory E2E,Pr Fcst,RO
CAST(NULL AS STRING) AS Site, --E2E
CAST(NULL AS STRING) AS Customer_ID, --E2E
CAST(NULL AS STRING) AS Project,--E2E
CAST(NULL AS STRING) AS Project_Type,--E2E
NULL AS Book_to_Bill,--Ass
NULL AS Estimated,--Ass
NULL AS No_of_Billing,--Ass
CAST(NULL AS FLOAT64) AS M1,--Ass
CAST(NULL AS FLOAT64) AS M2,--Ass
CAST(NULL AS FLOAT64) AS M3,--Ass
CAST(NULL AS FLOAT64) AS M4,--Ass
CAST(NULL AS FLOAT64) AS M5,--Ass
CAST(NULL AS FLOAT64) AS M6,--Ass
CAST(NULL AS FLOAT64) AS M7,--Ass
CAST(NULL AS FLOAT64) AS M8,--Ass
CAST(NULL AS FLOAT64) AS M9,--Ass
CAST(NULL AS FLOAT64) AS M10,--Ass
CAST(NULL AS FLOAT64) AS M11,--Ass
CAST(NULL AS FLOAT64) AS M12--Ass
FROM irm-fin-acct-dp-prod.IYR_Process.ROData_Clean)
''')

In [ ]:
_ = query_bq('''
CREATE OR REPLACE TABLE irm-fin-acct-dp-prod.IYR_Process.Union_Ins_Digital AS(
SELECT
Time_Stamp,
DATE_TRUNC(CURRENT_DATE(),WEEK(MONDAY)) AS Wk_Ending,
'B2B-E2E-Manual' as DataSource,
Region,
Cluster,
a.Country AS Country,
BU,
Product,
CAST(NULL AS STRING) AS SubProduct,
StoSer,
Segment,
RNR,
Phase,
Stage,
UID,
Opp_ID,
Opportunity_Name,
Deal_Size,
Age,
Account_Name,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
Probability,
Created_Date,
Close_Date,
First_Bill_Date,
Forecast_Category,
Annual_Amt,
EB AS MEB,
--Source,
SF1_Jan*(1-IFNULL(Tax.Tax,0)) AS SF1_Jan,
SF1_Feb*(1-IFNULL(Tax.Tax,0)) AS SF1_Feb,
SF1_Mar*(1-IFNULL(Tax.Tax,0)) AS SF1_Mar,
SF1_Apr*(1-IFNULL(Tax.Tax,0)) AS SF1_Apr,
SF1_May*(1-IFNULL(Tax.Tax,0)) AS SF1_May,
SF1_Jun*(1-IFNULL(Tax.Tax,0)) AS SF1_Jun,
SF1_Jul*(1-IFNULL(Tax.Tax,0)) AS SF1_Jul,
SF1_Aug*(1-IFNULL(Tax.Tax,0)) AS SF1_Aug,
SF1_Sep*(1-IFNULL(Tax.Tax,0)) AS SF1_Sep,
SF1_Oct*(1-IFNULL(Tax.Tax,0)) AS SF1_Oct,
SF1_Nov*(1-IFNULL(Tax.Tax,0)) AS SF1_Nov,
SF1_Dec*(1-IFNULL(Tax.Tax,0)) AS SF1_Dec,
SF1_FY*(1-IFNULL(Tax.Tax,0)) AS SF1_FY,
SF2_Jan*(1-IFNULL(Tax.Tax,0)) AS SF2_Jan,
SF2_Feb*(1-IFNULL(Tax.Tax,0)) AS SF2_Feb,
SF2_Mar*(1-IFNULL(Tax.Tax,0)) AS SF2_Mar,
SF2_Apr*(1-IFNULL(Tax.Tax,0)) AS SF2_Apr,
SF2_May*(1-IFNULL(Tax.Tax,0)) AS SF2_May,
SF2_Jun*(1-IFNULL(Tax.Tax,0)) AS SF2_Jun,
SF2_Jul*(1-IFNULL(Tax.Tax,0)) AS SF2_Jul,
SF2_Aug*(1-IFNULL(Tax.Tax,0)) AS SF2_Aug,
SF2_Sep*(1-IFNULL(Tax.Tax,0)) AS SF2_Sep,
SF2_Oct*(1-IFNULL(Tax.Tax,0)) AS SF2_Oct,
SF2_Nov*(1-IFNULL(Tax.Tax,0)) AS SF2_Nov,
SF2_Dec*(1-IFNULL(Tax.Tax,0)) AS SF2_Dec,
SF2_FY*(1-IFNULL(Tax.Tax,0)) AS SF2_FY,
EE1_Jan*(1-IFNULL(Tax.Tax,0)) AS EE1_Jan,
EE1_Feb*(1-IFNULL(Tax.Tax,0)) AS EE1_Feb,
EE1_Mar*(1-IFNULL(Tax.Tax,0)) AS EE1_Mar,
EE1_Apr*(1-IFNULL(Tax.Tax,0)) AS EE1_Apr,
EE1_May*(1-IFNULL(Tax.Tax,0)) AS EE1_May,
EE1_Jun*(1-IFNULL(Tax.Tax,0)) AS EE1_Jun,
EE1_Jul*(1-IFNULL(Tax.Tax,0)) AS EE1_Jul,
EE1_Aug*(1-IFNULL(Tax.Tax,0)) AS EE1_Aug,
EE1_Sep*(1-IFNULL(Tax.Tax,0)) AS EE1_Sep,
EE1_Oct*(1-IFNULL(Tax.Tax,0)) AS EE1_Oct,
EE1_Nov*(1-IFNULL(Tax.Tax,0)) AS EE1_Nov,
EE1_Dec*(1-IFNULL(Tax.Tax,0)) AS EE1_Dec,
EE1_FY*(1-IFNULL(Tax.Tax,0)) AS EE1_FY,
EE2_Jan*(1-IFNULL(Tax.Tax,0)) AS EE2_Jan,
EE2_Feb*(1-IFNULL(Tax.Tax,0)) AS EE2_Feb,
EE2_Mar*(1-IFNULL(Tax.Tax,0)) AS EE2_Mar,
EE2_Apr*(1-IFNULL(Tax.Tax,0)) AS EE2_Apr,
EE2_May*(1-IFNULL(Tax.Tax,0)) AS EE2_May,
EE2_Jun*(1-IFNULL(Tax.Tax,0)) AS EE2_Jun,
EE2_Jul*(1-IFNULL(Tax.Tax,0)) AS EE2_Jul,
EE2_Aug*(1-IFNULL(Tax.Tax,0)) AS EE2_Aug,
EE2_Sep*(1-IFNULL(Tax.Tax,0)) AS EE2_Sep,
EE2_Oct*(1-IFNULL(Tax.Tax,0)) AS EE2_Oct,
EE2_Nov*(1-IFNULL(Tax.Tax,0)) AS EE2_Nov,
EE2_Dec*(1-IFNULL(Tax.Tax,0)) AS EE2_Dec,
EE2_FY*(1-IFNULL(Tax.Tax,0)) AS EE2_FY,
--Comments,
Join_Flag   ,
CAST(NULL AS STRING) AS E2E_Match,--E2E
CAST(NULL AS STRING) AS Memo,--Product_Subcategory E2E,Pr Fcst,RO
CAST(NULL AS STRING) AS Site, --E2E
CAST(NULL AS STRING) AS Customer_ID, --E2E
CAST(NULL AS STRING) AS Project,--E2E
CAST(NULL AS STRING) AS Project_Type,--E2E
NULL AS Book_to_Bill,--Ass
NULL AS Estimated,--Ass
NULL AS No_of_Billing,--Ass
CAST(NULL AS FLOAT64) AS M1,--Ass
CAST(NULL AS FLOAT64) AS M2,--Ass
CAST(NULL AS FLOAT64) AS M3,--Ass
CAST(NULL AS FLOAT64) AS M4,--Ass
CAST(NULL AS FLOAT64) AS M5,--Ass
CAST(NULL AS FLOAT64) AS M6,--Ass
CAST(NULL AS FLOAT64) AS M7,--Ass
CAST(NULL AS FLOAT64) AS M8,--Ass
CAST(NULL AS FLOAT64) AS M9,--Ass
CAST(NULL AS FLOAT64) AS M10,--Ass
CAST(NULL AS FLOAT64) AS M11,--Ass
CAST(NULL AS FLOAT64) AS M12--Ass
FROM irm-fin-acct-dp-prod.IYR_Process.B2B_E2E_Join_Digital  a LEFT JOIN irm-fin-acct-dp-prod.IYR.Tax AS Tax ON a.Country = Tax.Country
)
''')

#### Trackers_Out

In [ ]:
_ = query_bq('''
CREATE OR REPLACE TABLE irm-fin-acct-dp-prod.IYR_Process.Trackers_Out AS
(WITH CTE AS (SELECT
Time_Stamp,
Wk_Ending,
DataSource,
Region,
Cluster,
Country,
BU,
Product,
SubProduct,
StoSer,
Segment,
RNR,
Phase,
Stage,
--SUBSTR(UID,20) AS UID,
CASE
WHEN Close_Date >= '2024-01-01' AND Close_Date <= '2024-06-30' THEN CONCAT(SUBSTR(UID,20),'-24H1')
WHEN Close_Date >= '2024-07-01' AND Close_Date <= '2024-12-31' THEN CONCAT(SUBSTR(UID,20),'-24H2')
ELSE CONCAT(SUBSTR(UID,20),'-',CAST(FORMAT_DATE('%y',Close_Date) AS STRING)) END AS UID,
CAST(NULL AS STRING) AS Opp_ID,
CAST(NULL AS STRING) AS Opportunity_Name,
Deal_Size,
NULL AS Age,
CAST(NULL AS STRING) AS Account_Name,
GI_Vertical,
CAST(NULL AS STRING) AS Deal_Type,
CAST(NULL AS STRING) AS Salesforce_URL,
CAST(NULL AS STRING) AS Opportunity_Owner,
Probability,
CAST(NULL AS DATE) AS Created_Date,
CASE
WHEN Close_Date >= '2024-01-01' AND Close_Date <= '2024-06-30' THEN CAST('2024-06-30' AS DATE)
WHEN Close_Date >= '2024-07-01' AND Close_Date <= '2024-12-31' THEN CAST('2024-12-31' AS DATE)
WHEN Close_Date >= '2025-01-01' AND Close_Date <= '2025-06-30' THEN CAST('2025-06-30' AS DATE)
WHEN Close_Date >= '2025-07-01' AND Close_Date <= '2025-12-31' THEN CAST('2025-12-31' AS DATE)
ELSE LAST_DAY(Close_Date,YEAR) END AS Close_Date,
CAST(NULL AS DATE) AS First_Bill_Date,
Forecast_Category,
Annual_Amt,
MEB,
SF1_Jan,
SF1_Feb,
SF1_Mar,
SF1_Apr,
SF1_May,
SF1_Jun,
SF1_Jul,
SF1_Aug,
SF1_Sep,
SF1_Oct,
SF1_Nov,
SF1_Dec,
SF1_FY,
SF2_Jan,
SF2_Feb,
SF2_Mar,
SF2_Apr,
SF2_May,
SF2_Jun,
SF2_Jul,
SF2_Aug,
SF2_Sep,
SF2_Oct,
SF2_Nov,
SF2_Dec,
SF2_FY,
EE1_Jan,
EE1_Feb,
EE1_Mar,
EE1_Apr,
EE1_May,
EE1_Jun,
EE1_Jul,
EE1_Aug,
EE1_Sep,
EE1_Oct,
EE1_Nov,
EE1_Dec,
EE1_FY,
EE2_Jan,
EE2_Feb,
EE2_Mar,
EE2_Apr,
EE2_May,
EE2_Jun,
EE2_Jul,
EE2_Aug,
EE2_Sep,
EE2_Oct,
EE2_Nov,
EE2_Dec,
EE2_FY,
Join_Flag,
E2E_Match,
Memo,
Site,
Customer_ID,
Project,
Project_Type,
Book_to_Bill,
Estimated,
No_of_Billing,
M1,
M2,
M3,
M4,
M5,
M6,
M7,
M8,
M9,
M10,
M11,
M12
FROM irm-fin-acct-dp-prod.IYR_Process.Union_Ins
WHERE (DataSource in ('B2B-E2E-Manual') AND Deal_Size = '< $25k') AND (Region = 'NA')),
CTE2 AS (SELECT
Time_Stamp,
Wk_Ending,
DataSource,
Region,
Cluster,
Country,
BU,
Product,
SubProduct,
StoSer,
Segment,
RNR,
Phase,
Stage,
UID,
Opp_ID,
Opportunity_Name,
Deal_Size,
Age,
Account_Name,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
Probability,
Created_Date,
Close_Date,
First_Bill_Date,
Forecast_Category,
SUM(Annual_Amt) AS Annual_Amt,
MEB,
SUM(SF1_Jan) AS SF1_Jan,
SUM(SF1_Feb) AS SF1_Feb,
SUM(SF1_Mar) AS SF1_Mar,
SUM(SF1_Apr) AS SF1_Apr,
SUM(SF1_May) AS SF1_May,
SUM(SF1_Jun) AS SF1_Jun,
SUM(SF1_Jul) AS SF1_Jul,
SUM(SF1_Aug) AS SF1_Aug,
SUM(SF1_Sep) AS SF1_Sep,
SUM(SF1_Oct) AS SF1_Oct,
SUM(SF1_Nov) AS SF1_Nov,
SUM(SF1_Dec) AS SF1_Dec,
SUM(SF1_FY) AS SF1_FY,
SUM(SF2_Jan) AS SF2_Jan,
SUM(SF2_Feb) AS SF2_Feb,
SUM(SF2_Mar) AS SF2_Mar,
SUM(SF2_Apr) AS SF2_Apr,
SUM(SF2_May) AS SF2_May,
SUM(SF2_Jun) AS SF2_Jun,
SUM(SF2_Jul) AS SF2_Jul,
SUM(SF2_Aug) AS SF2_Aug,
SUM(SF2_Sep) AS SF2_Sep,
SUM(SF2_Oct) AS SF2_Oct,
SUM(SF2_Nov) AS SF2_Nov,
SUM(SF2_Dec) AS SF2_Dec,
SUM(SF2_FY) AS SF2_FY,
SUM(EE1_Jan) AS EE1_Jan,
SUM(EE1_Feb) AS EE1_Feb,
SUM(EE1_Mar) AS EE1_Mar,
SUM(EE1_Apr) AS EE1_Apr,
SUM(EE1_May) AS EE1_May,
SUM(EE1_Jun) AS EE1_Jun,
SUM(EE1_Jul) AS EE1_Jul,
SUM(EE1_Aug) AS EE1_Aug,
SUM(EE1_Sep) AS EE1_Sep,
SUM(EE1_Oct) AS EE1_Oct,
SUM(EE1_Nov) AS EE1_Nov,
SUM(EE1_Dec) AS EE1_Dec,
SUM(EE1_FY) AS EE1_FY,
SUM(EE2_Jan) AS EE2_Jan,
SUM(EE2_Feb) AS EE2_Feb,
SUM(EE2_Mar) AS EE2_Mar,
SUM(EE2_Apr) AS EE2_Apr,
SUM(EE2_May) AS EE2_May,
SUM(EE2_Jun) AS EE2_Jun,
SUM(EE2_Jul) AS EE2_Jul,
SUM(EE2_Aug) AS EE2_Aug,
SUM(EE2_Sep) AS EE2_Sep,
SUM(EE2_Oct) AS EE2_Oct,
SUM(EE2_Nov) AS EE2_Nov,
SUM(EE2_Dec) AS EE2_Dec,
SUM(EE2_FY) AS EE2_FY,
Join_Flag,
E2E_Match,
Memo,
Site,
Customer_ID,
Project,
Project_Type,
Book_to_Bill,
Estimated,
No_of_Billing,
SUM(M1) AS M1,
SUM(M2) AS M2,
SUM(M3) AS M3,
SUM(M4) AS M4,
SUM(M5) AS M5,
SUM(M6) AS M6,
SUM(M7) AS M7,
SUM(M8) AS M8,
SUM(M9) AS M9,
SUM(M10)AS M10,
SUM(M11)AS M11,
SUM(M12) AS M12
FROM CTE
GROUP BY  --------------------------------
Time_Stamp,
Wk_Ending,
DataSource,
Region,
Cluster,
Country,
BU,
Product,
SubProduct,
StoSer,
Segment,
RNR,
Phase,
Stage,
UID,
Opp_ID,
Opportunity_Name,
Deal_Size,
Age,
Account_Name,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
Probability,
Created_Date,
Close_Date,
First_Bill_Date,
Forecast_Category,
MEB,
Join_Flag,
E2E_Match,
Memo,
Site,
Customer_ID,
Project,
Project_Type,
Book_to_Bill,
Estimated,
No_of_Billing

UNION ALL

SELECT
Time_Stamp,
Wk_Ending,
DataSource,
CASE WHEN Country = 'India' THEN 'India' ELSE Region END AS Region,
Cluster,
Country,
BU,
Product,
SubProduct,
StoSer,
Segment,
RNR,
Phase,
Stage,
UID,
Opp_ID,
Opportunity_Name,
Deal_Size,
Age,
Account_Name,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
Probability,
Created_Date,
Close_Date,
First_Bill_Date,
Forecast_Category,
Annual_Amt,
MEB,
SF1_Jan,
SF1_Feb,
SF1_Mar,
SF1_Apr,
SF1_May,
SF1_Jun,
SF1_Jul,
SF1_Aug,
SF1_Sep,
SF1_Oct,
SF1_Nov,
SF1_Dec,
SF1_FY,
SF2_Jan,
SF2_Feb,
SF2_Mar,
SF2_Apr,
SF2_May,
SF2_Jun,
SF2_Jul,
SF2_Aug,
SF2_Sep,
SF2_Oct,
SF2_Nov,
SF2_Dec,
SF2_FY,
EE1_Jan,
EE1_Feb,
EE1_Mar,
EE1_Apr,
EE1_May,
EE1_Jun,
EE1_Jul,
EE1_Aug,
EE1_Sep,
EE1_Oct,
EE1_Nov,
EE1_Dec,
EE1_FY,
EE2_Jan,
EE2_Feb,
EE2_Mar,
EE2_Apr,
EE2_May,
EE2_Jun,
EE2_Jul,
EE2_Aug,
EE2_Sep,
EE2_Oct,
EE2_Nov,
EE2_Dec,
EE2_FY,
Join_Flag,
E2E_Match,
Memo,
Site,
Customer_ID,
Project,
Project_Type,
Book_to_Bill,
Estimated,
No_of_Billing,
M1,
M2,
M3,
M4,
M5,
M6,
M7,
M8,
M9,
M10,
M11,
M12
FROM irm-fin-acct-dp-prod.IYR_Process.Union_Ins
WHERE NOT ((DataSource in ('B2B-E2E-Manual') AND Deal_Size = '< $25k') AND  (Region = 'NA')))
SELECT
Time_Stamp,
Wk_Ending,
DataSource,
CASE WHEN Region = 'IMES' THEN 'MAS' WHEN Product = 'IMES' THEN 'MAS' ELSE Region END as Region,
Cluster,
Country,
CASE WHEN Region = 'IMES' THEN 'Core' WHEN Product = 'IMES' THEN 'Core' ELSE BU END AS BU,
CASE WHEN Product = 'IMES' THEN 'MAS' ELSE Product END AS Product,
SubProduct,
StoSer,
Segment,
RNR,
Phase,
Stage,
UID,
Opp_ID,
Opportunity_Name,
Deal_Size,
Age,
Account_Name,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
Probability,
Created_Date,
Close_Date,
First_Bill_Date,
Forecast_Category,
Annual_Amt,
MEB,
SF1_Jan,
SF1_Feb,
SF1_Mar,
SF1_Apr,
SF1_May,
SF1_Jun,
SF1_Jul,
SF1_Aug,
SF1_Sep,
SF1_Oct,
SF1_Nov,
SF1_Dec,
SF1_FY,
SF2_Jan,
SF2_Feb,
SF2_Mar,
SF2_Apr,
SF2_May,
SF2_Jun,
SF2_Jul,
SF2_Aug,
SF2_Sep,
SF2_Oct,
SF2_Nov,
SF2_Dec,
SF2_FY,
EE1_Jan,
EE1_Feb,
EE1_Mar,
EE1_Apr,
EE1_May,
EE1_Jun,
EE1_Jul,
EE1_Aug,
EE1_Sep,
EE1_Oct,
EE1_Nov,
EE1_Dec,
EE1_FY,
EE2_Jan,
EE2_Feb,
EE2_Mar,
EE2_Apr,
EE2_May,
EE2_Jun,
EE2_Jul,
EE2_Aug,
EE2_Sep,
EE2_Oct,
EE2_Nov,
EE2_Dec,
EE2_FY,
Join_Flag,
E2E_Match,
Memo,
Site,
Customer_ID,
Project,
Project_Type,
Book_to_Bill,
Estimated,
No_of_Billing,
M1,
M2,
M3,
M4,
M5,
M6,
M7,
M8,
M9,
M10,
M11,
M12
FROM CTE2
ORDER BY DataSource ASC, Region ASC, Cluster ASC, Country ASC, BU ASC, Product ASC, StoSer ASC, Segment ASC, RNR ASC, Deal_Size ASC)
''')

In [ ]:
_ = query_bq('''
CREATE OR REPLACE TABLE irm-fin-acct-dp-prod.IYR_Process.Trackers_Out_Digital AS
(WITH CTE AS (SELECT
Time_Stamp,
Wk_Ending,
DataSource,
Region,
Cluster,
Country,
BU,
Product,
SubProduct,
StoSer,
Segment,
RNR,
Phase,
Stage,
--SUBSTR(UID,20) AS UID,
CASE
WHEN Close_Date >= '2024-01-01' AND Close_Date <= '2024-06-30' THEN CONCAT(SUBSTR(UID,20),'-24H1')
WHEN Close_Date >= '2024-07-01' AND Close_Date <= '2024-12-31' THEN CONCAT(SUBSTR(UID,20),'-24H2')
ELSE CONCAT(SUBSTR(UID,20),'-',CAST(FORMAT_DATE('%y',Close_Date) AS STRING)) END AS UID,
CAST(NULL AS STRING) AS Opp_ID,
CAST(NULL AS STRING) AS Opportunity_Name,
Deal_Size,
NULL AS Age,
CAST(NULL AS STRING) AS Account_Name,
GI_Vertical,
CAST(NULL AS STRING) AS Deal_Type,
CAST(NULL AS STRING) AS Salesforce_URL,
CAST(NULL AS STRING) AS Opportunity_Owner,
Probability,
CAST(NULL AS DATE) AS Created_Date,
CASE
WHEN Close_Date >= '2024-01-01' AND Close_Date <= '2024-06-30' THEN CAST('2024-06-30' AS DATE)
WHEN Close_Date >= '2024-07-01' AND Close_Date <= '2024-12-31' THEN CAST('2024-12-31' AS DATE)
WHEN Close_Date >= '2025-01-01' AND Close_Date <= '2025-06-30' THEN CAST('2025-06-30' AS DATE)
WHEN Close_Date >= '2025-07-01' AND Close_Date <= '2025-12-31' THEN CAST('2025-12-31' AS DATE)
ELSE LAST_DAY(Close_Date,YEAR) END AS Close_Date,
CAST(NULL AS DATE) AS First_Bill_Date,
Forecast_Category,
Annual_Amt,
MEB,
SF1_Jan,
SF1_Feb,
SF1_Mar,
SF1_Apr,
SF1_May,
SF1_Jun,
SF1_Jul,
SF1_Aug,
SF1_Sep,
SF1_Oct,
SF1_Nov,
SF1_Dec,
SF1_FY,
SF2_Jan,
SF2_Feb,
SF2_Mar,
SF2_Apr,
SF2_May,
SF2_Jun,
SF2_Jul,
SF2_Aug,
SF2_Sep,
SF2_Oct,
SF2_Nov,
SF2_Dec,
SF2_FY,
EE1_Jan,
EE1_Feb,
EE1_Mar,
EE1_Apr,
EE1_May,
EE1_Jun,
EE1_Jul,
EE1_Aug,
EE1_Sep,
EE1_Oct,
EE1_Nov,
EE1_Dec,
EE1_FY,
EE2_Jan,
EE2_Feb,
EE2_Mar,
EE2_Apr,
EE2_May,
EE2_Jun,
EE2_Jul,
EE2_Aug,
EE2_Sep,
EE2_Oct,
EE2_Nov,
EE2_Dec,
EE2_FY,
Join_Flag,
E2E_Match,
Memo,
Site,
Customer_ID,
Project,
Project_Type,
Book_to_Bill,
Estimated,
No_of_Billing,
M1,
M2,
M3,
M4,
M5,
M6,
M7,
M8,
M9,
M10,
M11,
M12
FROM irm-fin-acct-dp-prod.IYR_Process.Union_Ins_Digital
WHERE (DataSource in ('B2B-E2E-Manual') AND Deal_Size = '< $25k') AND (Region = 'NA')),
CTE2 AS (SELECT
Time_Stamp,
Wk_Ending,
DataSource,
Region,
Cluster,
Country,
BU,
Product,
SubProduct,
StoSer,
Segment,
RNR,
Phase,
Stage,
UID,
Opp_ID,
Opportunity_Name,
Deal_Size,
Age,
Account_Name,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
Probability,
Created_Date,
Close_Date,
First_Bill_Date,
Forecast_Category,
SUM(Annual_Amt) AS Annual_Amt,
MEB,
SUM(SF1_Jan) AS SF1_Jan,
SUM(SF1_Feb) AS SF1_Feb,
SUM(SF1_Mar) AS SF1_Mar,
SUM(SF1_Apr) AS SF1_Apr,
SUM(SF1_May) AS SF1_May,
SUM(SF1_Jun) AS SF1_Jun,
SUM(SF1_Jul) AS SF1_Jul,
SUM(SF1_Aug) AS SF1_Aug,
SUM(SF1_Sep) AS SF1_Sep,
SUM(SF1_Oct) AS SF1_Oct,
SUM(SF1_Nov) AS SF1_Nov,
SUM(SF1_Dec) AS SF1_Dec,
SUM(SF1_FY) AS SF1_FY,
SUM(SF2_Jan) AS SF2_Jan,
SUM(SF2_Feb) AS SF2_Feb,
SUM(SF2_Mar) AS SF2_Mar,
SUM(SF2_Apr) AS SF2_Apr,
SUM(SF2_May) AS SF2_May,
SUM(SF2_Jun) AS SF2_Jun,
SUM(SF2_Jul) AS SF2_Jul,
SUM(SF2_Aug) AS SF2_Aug,
SUM(SF2_Sep) AS SF2_Sep,
SUM(SF2_Oct) AS SF2_Oct,
SUM(SF2_Nov) AS SF2_Nov,
SUM(SF2_Dec) AS SF2_Dec,
SUM(SF2_FY) AS SF2_FY,
SUM(EE1_Jan) AS EE1_Jan,
SUM(EE1_Feb) AS EE1_Feb,
SUM(EE1_Mar) AS EE1_Mar,
SUM(EE1_Apr) AS EE1_Apr,
SUM(EE1_May) AS EE1_May,
SUM(EE1_Jun) AS EE1_Jun,
SUM(EE1_Jul) AS EE1_Jul,
SUM(EE1_Aug) AS EE1_Aug,
SUM(EE1_Sep) AS EE1_Sep,
SUM(EE1_Oct) AS EE1_Oct,
SUM(EE1_Nov) AS EE1_Nov,
SUM(EE1_Dec) AS EE1_Dec,
SUM(EE1_FY) AS EE1_FY,
SUM(EE2_Jan) AS EE2_Jan,
SUM(EE2_Feb) AS EE2_Feb,
SUM(EE2_Mar) AS EE2_Mar,
SUM(EE2_Apr) AS EE2_Apr,
SUM(EE2_May) AS EE2_May,
SUM(EE2_Jun) AS EE2_Jun,
SUM(EE2_Jul) AS EE2_Jul,
SUM(EE2_Aug) AS EE2_Aug,
SUM(EE2_Sep) AS EE2_Sep,
SUM(EE2_Oct) AS EE2_Oct,
SUM(EE2_Nov) AS EE2_Nov,
SUM(EE2_Dec) AS EE2_Dec,
SUM(EE2_FY) AS EE2_FY,
Join_Flag,
E2E_Match,
Memo,
Site,
Customer_ID,
Project,
Project_Type,
Book_to_Bill,
Estimated,
No_of_Billing,
SUM(M1) AS M1,
SUM(M2) AS M2,
SUM(M3) AS M3,
SUM(M4) AS M4,
SUM(M5) AS M5,
SUM(M6) AS M6,
SUM(M7) AS M7,
SUM(M8) AS M8,
SUM(M9) AS M9,
SUM(M10)AS M10,
SUM(M11)AS M11,
SUM(M12) AS M12
FROM CTE
GROUP BY  --------------------------------
Time_Stamp,
Wk_Ending,
DataSource,
Region,
Cluster,
Country,
BU,
Product,
SubProduct,
StoSer,
Segment,
RNR,
Phase,
Stage,
UID,
Opp_ID,
Opportunity_Name,
Deal_Size,
Age,
Account_Name,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
Probability,
Created_Date,
Close_Date,
First_Bill_Date,
Forecast_Category,
MEB,
Join_Flag,
E2E_Match,
Memo,
Site,
Customer_ID,
Project,
Project_Type,
Book_to_Bill,
Estimated,
No_of_Billing

UNION ALL

SELECT
Time_Stamp,
Wk_Ending,
DataSource,
CASE WHEN Country = 'India' THEN 'India' ELSE Region END AS Region,
Cluster,
Country,
BU,
Product,
SubProduct,
StoSer,
Segment,
RNR,
Phase,
Stage,
UID,
Opp_ID,
Opportunity_Name,
Deal_Size,
Age,
Account_Name,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
Probability,
Created_Date,
Close_Date,
First_Bill_Date,
Forecast_Category,
Annual_Amt,
MEB,
SF1_Jan,
SF1_Feb,
SF1_Mar,
SF1_Apr,
SF1_May,
SF1_Jun,
SF1_Jul,
SF1_Aug,
SF1_Sep,
SF1_Oct,
SF1_Nov,
SF1_Dec,
SF1_FY,
SF2_Jan,
SF2_Feb,
SF2_Mar,
SF2_Apr,
SF2_May,
SF2_Jun,
SF2_Jul,
SF2_Aug,
SF2_Sep,
SF2_Oct,
SF2_Nov,
SF2_Dec,
SF2_FY,
EE1_Jan,
EE1_Feb,
EE1_Mar,
EE1_Apr,
EE1_May,
EE1_Jun,
EE1_Jul,
EE1_Aug,
EE1_Sep,
EE1_Oct,
EE1_Nov,
EE1_Dec,
EE1_FY,
EE2_Jan,
EE2_Feb,
EE2_Mar,
EE2_Apr,
EE2_May,
EE2_Jun,
EE2_Jul,
EE2_Aug,
EE2_Sep,
EE2_Oct,
EE2_Nov,
EE2_Dec,
EE2_FY,
Join_Flag,
E2E_Match,
Memo,
Site,
Customer_ID,
Project,
Project_Type,
Book_to_Bill,
Estimated,
No_of_Billing,
M1,
M2,
M3,
M4,
M5,
M6,
M7,
M8,
M9,
M10,
M11,
M12
FROM irm-fin-acct-dp-prod.IYR_Process.Union_Ins_Digital
WHERE NOT ((DataSource in ('B2B-E2E-Manual') AND Deal_Size = '< $25k') AND  (Region = 'NA')))
SELECT
Time_Stamp,
Wk_Ending,
DataSource,
CASE WHEN Region = 'IMES' THEN 'MAS' WHEN Product = 'IMES' THEN 'MAS' ELSE Region END as Region,
Cluster,
Country,
CASE WHEN Region = 'IMES' THEN 'Core' WHEN Product = 'IMES' THEN 'Core' ELSE BU END AS BU,
CASE WHEN Product = 'IMES' THEN 'MAS' ELSE Product END AS Product,
SubProduct,
StoSer,
Segment,
RNR,
Phase,
Stage,
UID,
Opp_ID,
Opportunity_Name,
Deal_Size,
Age,
Account_Name,
GI_Vertical,
Deal_Type,
Salesforce_URL,
Opportunity_Owner,
Probability,
Created_Date,
Close_Date,
First_Bill_Date,
Forecast_Category,
Annual_Amt,
MEB,
SF1_Jan,
SF1_Feb,
SF1_Mar,
SF1_Apr,
SF1_May,
SF1_Jun,
SF1_Jul,
SF1_Aug,
SF1_Sep,
SF1_Oct,
SF1_Nov,
SF1_Dec,
SF1_FY,
SF2_Jan,
SF2_Feb,
SF2_Mar,
SF2_Apr,
SF2_May,
SF2_Jun,
SF2_Jul,
SF2_Aug,
SF2_Sep,
SF2_Oct,
SF2_Nov,
SF2_Dec,
SF2_FY,
EE1_Jan,
EE1_Feb,
EE1_Mar,
EE1_Apr,
EE1_May,
EE1_Jun,
EE1_Jul,
EE1_Aug,
EE1_Sep,
EE1_Oct,
EE1_Nov,
EE1_Dec,
EE1_FY,
EE2_Jan,
EE2_Feb,
EE2_Mar,
EE2_Apr,
EE2_May,
EE2_Jun,
EE2_Jul,
EE2_Aug,
EE2_Sep,
EE2_Oct,
EE2_Nov,
EE2_Dec,
EE2_FY,
Join_Flag,
E2E_Match,
Memo,
Site,
Customer_ID,
Project,
Project_Type,
Book_to_Bill,
Estimated,
No_of_Billing,
M1,
M2,
M3,
M4,
M5,
M6,
M7,
M8,
M9,
M10,
M11,
M12
FROM CTE2
ORDER BY DataSource ASC, Region ASC, Cluster ASC, Country ASC, BU ASC, Product ASC, StoSer ASC, Segment ASC, RNR ASC, Deal_Size ASC)
''')

#### Bookings_Out

In [ ]:
_ = query_bq('''
CREATE OR REPLACE TABLE irm-fin-acct-dp-prod.IYR_Process.Bookings_Out AS (
WITH BASE AS(SELECT Region, Cluster, Country, BU, Product, StoSer, Segment, RNR, Phase AS Stage, FORMAT_DATE('%b_%y',DATE_TRUNC(Close_Date,MONTH)) AS Period, Annual_amt
FROM irm-fin-acct-dp-prod.IYR_Process.Trackers_Out
WHERE DataSource = 'B2B-E2E-Manual' AND Close_Date >= '2024-01-01' AND Close_Date <= '2025-12-31' AND Close_Date IS NOT NULL),
PIVOTED AS (
SELECT *
FROM BASE
PIVOT (SUM(Annual_amt) FOR Period IN ('Jan_24', 'Feb_24', 'Mar_24', 'Apr_24', 'May_24', 'Jun_24', 'Jul_24', 'Aug_24', 'Sep_24', 'Oct_24', 'Nov_24', 'Dec_24', 'Jan_25', 'Feb_25', 'Mar_25', 'Apr_25', 'May_25', 'Jun_25', 'Jul_25', 'Aug_25', 'Sep_25', 'Oct_25', 'Nov_25', 'Dec_25')))
--SELECT Region, SUM(Annual_amt) FROM BASE GROUP BY 1 ORDER BY Region
--SELECT Region,SUM(IFNULL(Jan_24,0)+ IFNULL(Feb_24,0)+ IFNULL(Mar_24,0)+ IFNULL(Apr_24,0)+ IFNULL(May_24,0)+ IFNULL(Jun_24,0)+ IFNULL(Jul_24,0)+ IFNULL(Aug_24,0)+ IFNULL(Sep_24,0)+ IFNULL(Oct_24,0)+ IFNULL(Nov_24,0)+ IFNULL(Dec_24,0)+ IFNULL(Jan_25,0)+ IFNULL(Feb_25,0)+ IFNULL(Mar_25,0)+ IFNULL(Apr_25,0)+ IFNULL(May_25,0)+ IFNULL(Jun_25,0)+ IFNULL(Jul_25,0)+ IFNULL(Aug_25,0)+ IFNULL(Sep_25,0)+ IFNULL(Oct_25,0)+ IFNULL(Nov_25,0)+ IFNULL(Dec_25,0)) FROM PIVOTED GROUP BY 1 ORDER BY Region,
SELECT
Region,
Cluster,
Country,
BU,
Product,
StoSer,
Segment,
RNR,
Stage,
Jan_24,
Feb_24,
Mar_24,
Apr_24,
May_24,
Jun_24,
Jul_24,
Aug_24,
Sep_24,
Oct_24,
Nov_24,
Dec_24,
IFNULL(Jan_24,0)+ IFNULL(Feb_24,0)+ IFNULL(Mar_24,0)+ IFNULL(Apr_24,0)+ IFNULL(May_24,0)+ IFNULL(Jun_24,0)+ IFNULL(Jul_24,0)+ IFNULL(Aug_24,0)+ IFNULL(Sep_24,0)+ IFNULL(Oct_24,0)+ IFNULL(Nov_24,0)+ IFNULL(Dec_24,0) AS FY_24,
Jan_25,
Feb_25,
Mar_25,
Apr_25,
May_25,
Jun_25,
Jul_25,
Aug_25,
Sep_25,
Oct_25,
Nov_25,
Dec_25,
IFNULL(Jan_25,0)+ IFNULL(Feb_25,0)+ IFNULL(Mar_25,0)+ IFNULL(Apr_25,0)+ IFNULL(May_25,0)+ IFNULL(Jun_25,0)+ IFNULL(Jul_25,0)+ IFNULL(Aug_25,0)+ IFNULL(Sep_25,0)+ IFNULL(Oct_25,0)+ IFNULL(Nov_25,0)+ IFNULL(Dec_25,0) AS FY_25
FROM PIVOTED)
''')

## Read from templates

In [ ]:
workbook = gc.open_by_url('https://docs.google.com/spreadsheets/d/1SufGNEie5u3g6qdfEl7ultAFM_N_LI0HFIVDiScqLE0/edit#gid=1432790795')
worksheet = workbook.worksheet('TUrls')
data = worksheet.get_values(value_render_option = 'UNFORMATTED_VALUE')
urls = pd.DataFrame(columns=data[0],data=data[1:]).TempUrl.to_list()

In [ ]:
def get_df(url):
    print(url)
    workbook = gc.open_by_url(url)
    worksheet = workbook.worksheet('Bookings')
    data = worksheet.get_values('A1:CL',value_render_option = 'UNFORMATTED_VALUE')
    del data[0:3]
    data = list(filter(any,data))
    #_ = data.pop(0)
    headers = ['Region',
    'Cluster',
    'Country',
    'BU',
    'Product',
    'StoSer',
    'Segment',
    'RNR',
    'Stage',
    'UID',
    'Salesforce_URL',
    'Opp_ID',
    'Opportunity_Name',
    'Deal_Size',
    'Age',
    'Account_Name',
    'GI_Vertical',
    'Deal_Type',
    'Opportunity_Owner',
    'Created_Date',
    'Close_Date',
    'First_Bill_Date',
    'Probability',
    'Forecast_Category',
    'Annual_Amt',
    'MEB',
    'Source',
    'Blank0',
    'Blank1',
    'Blank2',
    'SF1_Jan',
    'SF1_Feb',
    'SF1_Mar',
    'SF1_Apr',
    'SF1_May',
    'SF1_Jun',
    'SF1_Jul',
    'SF1_Aug',
    'SF1_Sep',
    'SF1_Oct',
    'SF1_Nov',
    'SF1_Dec',
    'SF2_Jan',
    'SF2_Feb',
    'SF2_Mar',
    'SF2_Apr',
    'SF2_May',
    'SF2_Jun',
    'SF2_Jul',
    'SF2_Aug',
    'SF2_Sep',
    'SF2_Oct',
    'SF2_Nov',
    'SF2_Dec',
    'SF1_Q1',
    'SF1_Q2',
    'SF1_Q3',
    'SF1_Q4',
    'SF1_FY',
    'SF2_Q1',
    'SF2_Q2',
    'SF2_Q3',
    'SF2_Q4',
    'SF2_FY',
    'Blank3',
    'Comments',
    'M1_Jan',
    'M1_Feb',
    'M1_Mar',
    'M1_Apr',
    'M1_May',
    'M1_Jun',
    'M1_Jul',
    'M1_Aug',
    'M1_Sep',
    'M1_Oct',
    'M1_Nov',
    'M1_Dec',
    'M2_Jan',
    'M2_Feb',
    'M2_Mar',
    'M2_Apr',
    'M2_May',
    'M2_Jun',
    'M2_Jul',
    'M2_Aug',
    'M2_Sep',
    'M2_Oct',
    'M2_Nov',
    'M2_Dec']
    Bookings = pd.DataFrame(data, columns=headers).dropna(how='all')
    Bookings['URL'] = url
    Bookings['Source'] = Bookings['Source'].str.replace(' ','')
    Bookings['Sheet'] = 'Bookings'
    print('Bookings')

    workbook = gc.open_by_url(url)
    worksheet = workbook.worksheet('Pipeline')
    data = worksheet.get_values('A1:CL',value_render_option = 'UNFORMATTED_VALUE')
    del data[0:3]
    data = list(filter(any,data))
    headers = ['Region',
    'Cluster',
    'Country',
    'BU',
    'Product',
    'StoSer',
    'Segment',
    'RNR',
    'Stage',
    'UID',
    'Salesforce_URL',
    'Opp_ID',
    'Opportunity_Name',
    'Deal_Size',
    'Age',
    'Account_Name',
    'GI_Vertical',
    'Deal_Type',
    'Opportunity_Owner',
    'Created_Date',
    'Close_Date',
    'First_Bill_Date',
    'Probability',
    'Forecast_Category',
    'Annual_Amt',
    'MEB',
    'Source',
    'Blank0',
    'Blank1',
    'Blank2',
    'SF1_Jan',
    'SF1_Feb',
    'SF1_Mar',
    'SF1_Apr',
    'SF1_May',
    'SF1_Jun',
    'SF1_Jul',
    'SF1_Aug',
    'SF1_Sep',
    'SF1_Oct',
    'SF1_Nov',
    'SF1_Dec',
    'SF2_Jan',
    'SF2_Feb',
    'SF2_Mar',
    'SF2_Apr',
    'SF2_May',
    'SF2_Jun',
    'SF2_Jul',
    'SF2_Aug',
    'SF2_Sep',
    'SF2_Oct',
    'SF2_Nov',
    'SF2_Dec',
    'SF1_Q1',
    'SF1_Q2',
    'SF1_Q3',
    'SF1_Q4',
    'SF1_FY',
    'SF2_Q1',
    'SF2_Q2',
    'SF2_Q3',
    'SF2_Q4',
    'SF2_FY',
    'Blank3',
    'Comments',
    'M1_Jan',
    'M1_Feb',
    'M1_Mar',
    'M1_Apr',
    'M1_May',
    'M1_Jun',
    'M1_Jul',
    'M1_Aug',
    'M1_Sep',
    'M1_Oct',
    'M1_Nov',
    'M1_Dec',
    'M2_Jan',
    'M2_Feb',
    'M2_Mar',
    'M2_Apr',
    'M2_May',
    'M2_Jun',
    'M2_Jul',
    'M2_Aug',
    'M2_Sep',
    'M2_Oct',
    'M2_Nov',
    'M2_Dec']
    Pipeline = pd.DataFrame(data, columns=headers).dropna(how='all')
    Pipeline['URL'] = url
    Pipeline['Source'] = Pipeline['Source'].str.replace(' ','')
    Pipeline['Sheet'] = 'Pipeline'
    print('Pipeline')

    workbook = gc.open_by_url(url)
    worksheet = workbook.worksheet('IYR Overlay')
    data = worksheet.get_values('A1:AS',value_render_option = 'UNFORMATTED_VALUE')
    del data[0:3]
    data = list(filter(any,data))
    headers_P = ['Region',
    'Cluster',
    'Country',
    'BU',
    'Product',
    'StoSer',
    'Segment',
    'RNR',
    'Stage',
    'UID',
    'Comments',
    'SF1_Jan',
    'SF1_Feb',
    'SF1_Mar',
    'SF1_Apr',
    'SF1_May',
    'SF1_Jun',
    'SF1_Jul',
    'SF1_Aug',
    'SF1_Sep',
    'SF1_Oct',
    'SF1_Nov',
    'SF1_Dec',
    'SF2_Jan',
    'SF2_Feb',
    'SF2_Mar',
    'SF2_Apr',
    'SF2_May',
    'SF2_Jun',
    'SF2_Jul',
    'SF2_Aug',
    'SF2_Sep',
    'SF2_Oct',
    'SF2_Nov',
    'SF2_Dec',
    'SF1_Q1',
    'SF1_Q2',
    'SF1_Q3',
    'SF1_Q4',
    'SF1_FY',
    'SF2_Q1',
    'SF2_Q2',
    'SF2_Q3',
    'SF2_Q4',
    'SF2_FY']
    Forecast = pd.DataFrame(data, columns=headers_P).dropna(how='all')
    Forecast['Source'] = 'Manual OL'
    Forecast['MEB'] = 'M'
    Forecast['Sheet'] = 'Forecast'
    Forecast['URL'] = url
    for i in [x for x in headers if x not in headers_P]:
        Forecast[i] = np.nan
    print('Forecast')
    Forecast = Forecast[headers+['URL','Sheet']]
    return pd.concat([Forecast,Bookings,Pipeline])

In [ ]:
df_final = pd.concat([get_df(i) for i in urls]).reset_index(drop=True)
df_final.drop(columns=['Blank0','Blank1','Blank2','Blank3'],inplace=True)
df_final.replace('',np.nan,inplace=True)

https://docs.google.com/spreadsheets/d/1bwrCQpJ8isFPFqYC6j9BbRSr3DNPRQYhdnuEG3cI9Ws/edit?usp=drive_link
Bookings
Pipeline
Forecast
https://docs.google.com/spreadsheets/d/1rxlRbZzD7cJmZFTROYJsScSRuJsCTNo1A_Bs5Kb5oKU/edit?gid=1831153224#gid=1831153224
Bookings
Pipeline
Forecast
https://docs.google.com/spreadsheets/d/1c1kVNNuf8yuNzjU5qcY5AACgY5G2TroPTEzIbbwF9UU/edit?gid=1827907307#gid=1827907307


<ipython-input-4-a071e7e01eff>:265: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([Forecast,Bookings,Pipeline])


Bookings
Pipeline
Forecast
https://docs.google.com/spreadsheets/d/1a6u-GHrDDbz7rixy54v-2uJ9w2GnjUfldXj42oUq5aA/edit?usp=drive_web&ouid=115121684477242582764


<ipython-input-4-a071e7e01eff>:265: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([Forecast,Bookings,Pipeline])


Bookings
Pipeline
Forecast
https://docs.google.com/spreadsheets/d/114ETbh2NNmzFM7_FXtPgrupPS57lm1BVs6MErkmk1tQ/edit?gid=1831153224#gid=1831153224
Bookings
Pipeline
Forecast
https://docs.google.com/spreadsheets/d/1Fn-Bb9p1_fD004ycvyhx6QZsecwfQ7vCPVXGyGAWaQI/edit?gid=1831153224#gid=1831153224
Bookings
Pipeline
Forecast
https://docs.google.com/spreadsheets/d/1kalqNmCiL1TpiR0rfhSONkTB-goOLYrndIHvzwhQcto/edit?gid=1831153224#gid=1831153224
Bookings
Pipeline
Forecast
https://docs.google.com/spreadsheets/d/1xE3Cvwd914h3fClhLK28vbLEHPonUABBbbleRgWYjvA/edit?gid=1831153224#gid=1831153224
Bookings
Pipeline
Forecast
https://docs.google.com/spreadsheets/d/15Qfo1KeWSu9rziQ_gzu1kWsTnA_OwAKVX6y_i60JMqU/edit?gid=1831153224#gid=1831153224
Bookings
Pipeline
Forecast
https://docs.google.com/spreadsheets/d/1fp07LH0ai6Qpv0bcm4U1M_6CLDHuqhp8jX-HlsgxB2w/edit?gid=1831153224#gid=1831153224
Bookings
Pipeline
Forecast
https://docs.google.com/spreadsheets/d/1kO-unhCZvtueaVTjEA1JeGZLhIS9ssfbHHzPm34kJ7I/edit?gid=1831

...

<ipython-input-4-a071e7e01eff>:265: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([Forecast,Bookings,Pipeline])


Bookings
Pipeline
Forecast
https://docs.google.com/spreadsheets/d/1-9-ZhckdWzdwciWqkjYRjFksFvbe0FX7v2y4AIKVoDs/edit?gid=0#gid=0


<ipython-input-4-a071e7e01eff>:265: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([Forecast,Bookings,Pipeline])


Bookings
Pipeline
Forecast


<ipython-input-4-a071e7e01eff>:265: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([Forecast,Bookings,Pipeline])
<ipython-input-5-d9d518bdbc73>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final.replace('',np.nan,inplace=True)


In [ ]:
df_final.loc[df_final['Probability'].isna() ,'Probability'] = '0'
df_final['Probability'] = df_final['Probability'].str.replace('%','').astype('float')/100
df_final['Age'] = df_final['Age'].fillna('0').replace('','0').astype('int')

<ipython-input-6-0321ed4c844c>:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[df_final['Probability'].isna() ,'Probability'] = '0'


In [ ]:
df_final.replace('-','0',inplace=True)
df_final.replace('#N/A-Argumentmustbearange.','0',inplace=True)
df_final.replace('#N/A (Argument must be a range.)','0',inplace=True)

In [ ]:
floats = ['Annual_Amt','SF1_Jan', 'SF1_Feb', 'SF1_Mar', 'SF1_Apr', 'SF1_May', 'SF1_Jun',
       'SF1_Jul', 'SF1_Aug', 'SF1_Sep', 'SF1_Oct', 'SF1_Nov', 'SF1_Dec',
       'SF2_Jan', 'SF2_Feb', 'SF2_Mar', 'SF2_Apr', 'SF2_May', 'SF2_Jun',
       'SF2_Jul', 'SF2_Aug', 'SF2_Sep', 'SF2_Oct', 'SF2_Nov', 'SF2_Dec',
       'SF1_Q1', 'SF1_Q2', 'SF1_Q3', 'SF1_Q4', 'SF1_FY', 'SF2_Q1', 'SF2_Q2',
       'SF2_Q3', 'SF2_Q4', 'SF2_FY', 'M1_Jan', 'M1_Feb', 'M1_Mar',
       'M1_Apr', 'M1_May', 'M1_Jun', 'M1_Jul', 'M1_Aug', 'M1_Sep', 'M1_Oct',
       'M1_Nov', 'M1_Dec', 'M2_Jan', 'M2_Feb', 'M2_Mar', 'M2_Apr', 'M2_May',
       'M2_Jun', 'M2_Jul', 'M2_Aug', 'M2_Sep', 'M2_Oct', 'M2_Nov', 'M2_Dec']
for i in floats:
    if str(df_final[i].dtype) == 'object':
        df_final.loc[df_final[i] == '',i] = 0
        df_final[i] = pd.to_numeric(df_final[i], errors='coerce')
    df_final[i] = df_final[i].astype('float')
    df_final[i] = df_final[i].fillna(0.0)

In [ ]:
def getTimestamp(x):
    try:
        return
    except Exception:
        return np.nan

In [ ]:
dates = ['Created_Date','Close_Date','First_Bill_Date']
for i in dates:
    if str(df_final[i].dtype.type) != "<class 'numpy.datetime64'>":
        print(i)
        df_final.loc[df_final[i].notna(),i] = ((df_final.loc[df_final[i].notna(),i] - 25569) * 86400).apply(pd.Timestamp.fromtimestamp)
        df_final[i] = pd.to_datetime(df_final[i])

Created_Date


<ipython-input-10-cb11b73de9c6>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<DatetimeArray>
['2024-10-02 00:00:00', '2024-10-02 00:00:00', '2024-09-27 00:00:00',
 '2024-09-25 00:00:00', '2024-09-19 00:00:00', '2024-09-19 00:00:00',
 '2024-09-17 00:00:00', '2024-09-17 00:00:00', '2024-09-17 00:00:00',
 '2024-09-16 00:00:00',
 ...
 '2021-05-11 00:00:00', '2021-04-29 00:00:00', '2021-03-10 00:00:00',
 '2021-02-25 00:00:00', '2021-02-25 00:00:00', '2021-02-08 00:00:00',
 '2021-02-08 00:00:00', '2020-04-02 00:00:00', '2018-08-13 00:00:00',
 '2018-05-04 00:00:00']
Length: 77413, dtype: datetime64[ns]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[df_final[i].notna(),i] = ((df_final.loc[df_final[i].notna(),i] - 25569) * 86400).apply(pd.Timestamp.fromtimestamp)


Close_Date


<ipython-input-10-cb11b73de9c6>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<DatetimeArray>
['2024-10-02 00:00:00', '2024-10-04 00:00:00', '2024-09-30 00:00:00',
 '2024-09-25 00:00:00', '2024-10-01 00:00:00', '2024-09-30 00:00:00',
 '2024-09-24 00:00:00', '2024-09-25 00:00:00', '2024-09-19 00:00:00',
 '2024-09-25 00:00:00',
 ...
 '2025-01-31 00:00:00', '2025-02-21 00:00:00', '2025-01-24 00:00:00',
 '2024-10-25 00:00:00', '2024-10-25 00:00:00', '2024-10-31 00:00:00',
 '2024-10-31 00:00:00', '2025-02-15 00:00:00', '2024-12-17 00:00:00',
 '2025-01-15 00:00:00']
Length: 77605, dtype: datetime64[ns]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[df_final[i].notna(),i] = ((df_final.loc[df_final[i].notna(),i] - 25569) * 86400).apply(pd.Timestamp.fromtimestamp)


First_Bill_Date


<ipython-input-10-cb11b73de9c6>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '<DatetimeArray>
['2024-10-31 00:00:00', '2024-11-08 00:00:00', '2024-11-01 00:00:00',
 '2024-10-29 00:00:00', '2024-11-30 00:00:00', '2024-10-30 00:00:00',
 '2024-10-31 00:00:00', '2024-10-01 00:00:00', '2024-11-01 00:00:00',
 '2024-10-22 00:00:00',
 ...
 '2025-02-28 00:00:00', '2025-03-28 00:00:00', '2025-02-21 00:00:00',
 '2024-12-20 00:00:00', '2024-12-20 00:00:00', '2024-11-29 00:00:00',
 '2024-11-29 00:00:00', '2025-03-22 00:00:00', '2025-01-28 00:00:00',
 '2025-02-27 00:00:00']
Length: 73696, dtype: datetime64[ns]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_final.loc[df_final[i].notna(),i] = ((df_final.loc[df_final[i].notna(),i] - 25569) * 86400).apply(pd.Timestamp.fromtimestamp)


In [ ]:
df_final['First_Bill_Date'] = pd.to_datetime(df_final['First_Bill_Date'],errors = 'coerce')

In [ ]:
df_final['Comments'] = df_final['Comments'].astype('string')

In [ ]:
print((df_final.eval('SF1_Jan+SF1_Feb+SF1_Mar+SF1_Apr+SF1_May+SF1_Jun+SF1_Jul+SF1_Aug+SF1_Sep+SF1_Oct+SF1_Nov+SF1_Dec')).sum())
print(df_final.eval('SF1_Q1+SF1_Q2+SF1_Q3+SF1_Q4').sum())
print(df_final['SF1_FY'].sum())
print('%%%%%%%%%%%%')
print(df_final.eval('SF2_Jan+SF2_Feb+SF2_Mar+SF2_Apr+SF2_May+SF2_Jun+SF2_Jul+SF2_Aug+SF2_Sep+SF2_Oct+SF2_Nov+SF2_Dec').sum())
print(df_final.eval('SF2_Q1+SF2_Q2+SF2_Q3+SF2_Q4').sum())
print(df_final['SF2_FY'].sum())

644018672.3688406
644018672.3688406
644018672.3688406
%%%%%%%%%%%%
586672781.5673056
586672781.5673056
586672781.5673056


In [ ]:
upload_df_bq(df_final,"irm-fin-acct-dp-prod",'IYR_Process.Trackers_In')

100%|██████████| 1/1 [00:00<00:00, 5329.48it/s]


In [ ]:
_ = query_bq("""
CREATE OR REPLACE TABLE irm-fin-acct-dp-prod.IYR_Process.REP_24Carryover AS (
SELECT Region,Cluster,Country,BU,Product,StoSer,Segment,RNR,CASE
WHEN SAFE_CAST(LEFT(Stage,1) AS INTEGER)<= 4  OR (Stage = 'Pipeline') THEN 'Pipeline'
WHEN (SAFE_CAST(LEFT(Stage,1) AS INTEGER)>= 5  AND SAFE_CAST(LEFT(Stage,1) AS INTEGER)<= 7) OR (Stage = 'Bookings') THEN 'Bookings'
ELSE NULL END AS Phase,Sheet,
CASE WHEN Close_Date >= '2024-01-01' AND Close_Date < '2024-07-01' THEN '1H24'
WHEN Close_Date >= '2024-07-01' AND Close_Date < '2025-01-01' THEN '2H24' END AS CloseH,
SUM(IFNULL(SF1_Jan,0)) AS Jan_24,
SUM(IFNULL(SF1_Feb,0)) AS Feb_24,
SUM(IFNULL(SF1_Mar,0)) AS Mar_24,
SUM(IFNULL(SF1_Apr,0)) AS Apr_24,
SUM(IFNULL(SF1_May,0)) AS May_24,
SUM(IFNULL(SF1_Jun,0)) AS Jun_24,
SUM(IFNULL(SF1_Jul,0)) AS Jul_24,
SUM(IFNULL(SF1_Aug,0)) AS Aug_24,
SUM(IFNULL(SF1_Sep,0)) AS Sep_24,
SUM(IFNULL(SF1_Oct,0)) AS Oct_24,
SUM(IFNULL(SF1_Nov,0)) AS Nov_24,
SUM(IFNULL(SF1_Dec,0)) AS Dec_24,
SUM(IFNULL(SF2_Jan,0)) AS Jan_25,
SUM(IFNULL(SF2_Feb,0)) AS Feb_25,
SUM(IFNULL(SF2_Mar,0)) AS Mar_25,
SUM(IFNULL(SF2_Apr,0)) AS Apr_25,
SUM(IFNULL(SF2_May,0)) AS May_25,
SUM(IFNULL(SF2_Jun,0)) AS Jun_25,
SUM(IFNULL(SF2_Jul,0)) AS Jul_25,
SUM(IFNULL(SF2_Aug,0)) AS Aug_25,
SUM(IFNULL(SF2_Sep,0)) AS Sep_25,
SUM(IFNULL(SF2_Oct,0)) AS Oct_25,
SUM(IFNULL(SF2_Nov,0)) AS Nov_25,
SUM(IFNULL(SF2_Dec,0)) AS Dec_25,
SUM(IFNULL(SF1_Q1,0)) AS Q1_24,
SUM(IFNULL(SF1_Q2,0)) AS Q2_24,
SUM(IFNULL(SF1_Q3,0)) AS Q3_24,
SUM(IFNULL(SF1_Q4,0)) AS Q4_24,
SUM(IFNULL(SF1_FY,0)) AS FY_24,
SUM(IFNULL(SF2_Q1,0)) AS Q1_25,
SUM(IFNULL(SF2_Q2,0)) AS Q2_25,
SUM(IFNULL(SF2_Q3,0)) AS Q3_25,
SUM(IFNULL(SF2_Q4,0)) AS Q4_25,
SUM(IFNULL(SF2_FY,0)) AS FY_25,
FROM irm-fin-acct-dp-prod.IYR_Process.Trackers_In
WHERE Close_Date >= '2024-01-01' AND Close_Date < '2025-01-01'
GROUP BY ALL)
""")